In [1]:
!pip install boto3
!pip install rouge-score evaluate -q
!pip install bert_score
!pip -q install faiss-cpu sentence-transformers
!pip install openai
!pip install gradio

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable


In [2]:
import io
import os
import re
import math
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
from datetime import datetime
from typing import Dict, Any, List, Tuple, Optional
from openai import OpenAI
import getpass
import boto3

#AWS S3 ACCESS

Our S3 bucket, `nyu-tandon-cs-gy-6513-project`, contains approximately 1,300+ datasets - including 200+ datasets downloaded via Auctus and 1,100+ datasets from NYC Open Data.

The bucket is organized into two top-level directories: `auctus` and `nyc_open_data`.


**Auctus directory**

The auctus directory contains datasets downloaded from the Auctus platform and is divided into two subdirectories:

- `auctus/dataset_no_description` — datasets that do not have descriptions (because descriptions were not available via Auctus).

- `auctus/dataset_has_description` — datasets with descriptions. Each dataset in this directory has a corresponding JSON file containing its description summary, using the same base filename as the dataset.

**NYC Open Data directory**

The `nyc_open_data` directory contains all datasets downloaded from NYC Open Data. These datasets all include descriptions, which are consolidated in a single file named `dataset_catalog.csv`.

In [ ]:
import os

os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""

os.environ["OPENAI_API_KEY"] = ""
os.environ["HF_TOKEN"] = ""

openaimodel = "gpt-4.1-2025-04-14"

API KEYS

In [ ]:
import os
import getpass

try:
    from dotenv import load_dotenv
    load_dotenv()
    HAVE_DOTENV = True
except ImportError:
    HAVE_DOTENV = False


def ensure_env_local(env_name, prompt_text=None):
    """
    Ensure an environment variable is set for local notebooks:
    1) If already in os.environ, keep it.
    2) Else, if python-dotenv is available, attempt to load from .env.
    3) Else, prompt user securely via getpass.
    """
    if os.environ.get(env_name):
        print(f"{env_name} already set in environment.")
        return

    # If dotenv exists, it may have been loaded above.
    if HAVE_DOTENV and os.environ.get(env_name):
        print(f"🔐 Loaded {env_name} from .env.")
        return

    if prompt_text is None:
        prompt_text = f"Enter {env_name}: "

    os.environ[env_name] = getpass.getpass(prompt_text)
    print(f"Set {env_name} from manual input.")


ensure_env_local("AWS_ACCESS_KEY_ID")
ensure_env_local("AWS_SECRET_ACCESS_KEY")
ensure_env_local("OPENAI_API_KEY")

# Model selection
openaimodel = "gpt-4.1-2025-04-14"


AWS_ACCESS_KEY_ID already set in environment.
AWS_SECRET_ACCESS_KEY already set in environment.
OPENAI_API_KEY already set in environment.


In [5]:
# Initialize S3 client
s3 = boto3.client(
    "s3",
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
    region_name="us-east-1"
)

In [6]:
response = s3.list_objects_v2(Bucket="nyu-tandon-cs-gy-6513-project")

for obj in response.get("Contents", []):
    print(obj["Key"])

auctus/
auctus/dataset_descriptions/2007_-_2008_School_Progress_Reports_-_All_Schools_description.json
auctus/dataset_descriptions/Atomic_Polygons_description.json
auctus/dataset_descriptions/Ballfields_9K_description.json
auctus/dataset_descriptions/Bike_Ridership_-_Edmonton_Insight_Community_description.json
auctus/dataset_descriptions/Chicago_Park_District_Activities_description.json
auctus/dataset_descriptions/Chicago_Park_District__Movies_in_the_Parks_2014_description.json
auctus/dataset_descriptions/Chicago_Park_District__Movies_in_the_Parks_2016_description.json
auctus/dataset_descriptions/Chicago_Park_District__Movies_in_the_Parks_2017_description.json
auctus/dataset_descriptions/Chicago_Park_District__Movies_in_the_Parks_2018_description.json
auctus/dataset_descriptions/Chicago_Public_Schools_-_School_Admissions_Information_SY1819_description.json
auctus/dataset_descriptions/Chicago_Public_Schools_-_School_Admissions_Information_SY2324_description.json
auctus/dataset_descripti

#DATA PROFILER

In [ ]:
"""
=== Helper functions ===
This cell defines small utility functions used by the profiler:
"""

import warnings

def _try_to_datetime(series: pd.Series) -> Tuple[pd.Series, bool]:
    if pd.api.types.is_datetime64_any_dtype(series):
        return series, True
    try:
        with warnings.catch_warnings():
            warnings.filterwarnings(
                "ignore",
                message="Could not infer format, so each element will be parsed individually.*",
                category=UserWarning,
            )
            s = pd.to_datetime(series, errors="raise")
        return s, True
    except Exception:
        return series, False

def _safe_quantiles(s: pd.Series, qs=(0.05, 0.25, 0.5, 0.75, 0.95)) -> Dict[str, float]:
    try:
        q = s.quantile(qs)
        return {f"q{int(p*100)}": float(v) for p, v in q.items() if pd.notna(v)}
    except Exception:
        return {}

def _histogram(s: pd.Series, bins: int = 10) -> Dict[str, Any]:
    arr = pd.to_numeric(s, errors="coerce").dropna().to_numpy()
    if arr.size == 0 or np.nanmax(arr) == np.nanmin(arr):
        return {"bins": [], "counts": []}
    counts, edges = np.histogram(arr, bins=bins)
    centers = (edges[:-1] + edges[1:]) / 2.0
    return {"bins": centers.tolist(), "counts": counts.astype(int).tolist()}

def _top_values(s: pd.Series, k: int = 10) -> List[Dict[str, Any]]:
    vc = s.value_counts(dropna=True).head(k)
    total = int(s.notna().sum())
    return [{"value": str(v), "count": int(c), "pct": float(c/total) if total else 0.0}
            for v, c in vc.items()]

def _text_stats(s: pd.Series) -> Dict[str, Any]:
    s2 = s.dropna().astype(str)
    if s2.empty:
        return {"avg_length": None, "q25_length": None, "q50_length": None, "q75_length": None}
    lens = s2.str.len()
    return {
        "avg_length": float(lens.mean()),
        "q25_length": float(lens.quantile(0.25)),
        "q50_length": float(lens.quantile(0.50)),
        "q75_length": float(lens.quantile(0.75)),
    }


In [ ]:
# --- JIT-accelerated numeric stats for profiling ---
from numba import njit
import numpy as np

@njit
def _nanmean_numba(x):
    total = 0.0
    count = 0
    for i in range(x.shape[0]):
        v = x[i]
        if not np.isnan(v):
            total += v
            count += 1
    if count == 0:
        return np.nan
    return total / count

@njit
def _nanstd_numba(x):
    mean = _nanmean_numba(x)
    if np.isnan(mean):
        return np.nan
    total = 0.0
    count = 0
    for i in range(x.shape[0]):
        v = x[i]
        if not np.isnan(v):
            diff = v - mean
            total += diff * diff
            count += 1
    if count <= 1:
        return np.nan
    return np.sqrt(total / (count - 1))

@njit
def _nanquantile_sorted(sorted_x, q):
    n = sorted_x.shape[0]
    n_non_nan = n
    # ignore trailing NaNs in a sorted array
    for i in range(n - 1, -1, -1):
        if np.isnan(sorted_x[i]):
            n_non_nan -= 1
        else:
            break
    if n_non_nan == 0:
        return np.nan

    pos = q * (n_non_nan - 1)
    lo = int(np.floor(pos))
    hi = int(np.ceil(pos))
    if lo == hi:
        return sorted_x[lo]
    w = pos - lo
    return sorted_x[lo] * (1.0 - w) + sorted_x[hi] * w

@njit
def _nan_quantiles_numba(x):
    arr = x.copy()
    arr.sort()
    q25 = _nanquantile_sorted(arr, 0.25)
    q50 = _nanquantile_sorted(arr, 0.50)
    q75 = _nanquantile_sorted(arr, 0.75)
    return q25, q50, q75

@njit
def _histogram_numba(x, bins):
    # simple histogram ignoring NaNs
    min_val = np.inf
    max_val = -np.inf
    for i in range(x.shape[0]):
        v = x[i]
        if not np.isnan(v):
            if v < min_val:
                min_val = v
            if v > max_val:
                max_val = v

    if not np.isfinite(min_val) or not np.isfinite(max_val):
        return np.empty(0, dtype=np.float64), np.empty(0, dtype=np.int64)

    edges = np.linspace(min_val, max_val, bins + 1)
    counts = np.zeros(bins, dtype=np.int64)

    for i in range(x.shape[0]):
        v = x[i]
        if np.isnan(v):
            continue
        if v == max_val:
            idx = bins - 1
        else:
            idx = int((v - min_val) / (max_val - min_val) * bins)
            if idx < 0:
                idx = 0
            elif idx >= bins:
                idx = bins - 1
        counts[idx] += 1

    centers = (edges[:-1] + edges[1:]) * 0.5
    return centers, counts

@njit
def _numeric_summary_numba(x, bins):
    min_val = np.inf
    max_val = -np.inf
    for i in range(x.shape[0]):
        v = x[i]
        if not np.isnan(v):
            if v < min_val:
                min_val = v
            if v > max_val:
                max_val = v
    if not np.isfinite(min_val) or not np.isfinite(max_val):
        min_val = np.nan
        max_val = np.nan

    mean_val = _nanmean_numba(x)
    std_val = _nanstd_numba(x)
    q25, q50, q75 = _nan_quantiles_numba(x)
    centers, counts = _histogram_numba(x, bins)
    return min_val, max_val, mean_val, std_val, q25, q50, q75, centers, counts

def numeric_stats_from_series(s: pd.Series, hist_bins: int = 10) -> Dict[str, Any]:
    """
    Wrapper that bridges pandas -> jitted NumPy, and back into a JSON-serializable dict.
    """
    s_num = pd.to_numeric(s, errors="coerce")
    arr = s_num.to_numpy(dtype=np.float64)

    min_v, max_v, mean_v, std_v, q25, q50, q75, centers, counts = _numeric_summary_numba(
        arr, hist_bins
    )

    return {
        "min": None if np.isnan(min_v) else float(min_v),
        "max": None if np.isnan(max_v) else float(max_v),
        "mean": None if np.isnan(mean_v) else float(mean_v),
        "std": None if np.isnan(std_v) else float(std_v),
        "quantiles": {
            "q25": None if np.isnan(q25) else float(q25),
            "q50": None if np.isnan(q50) else float(q50),
            "q75": None if np.isnan(q75) else float(q75),
        },
        "histogram": {
            "bin_centers": [float(c) for c in centers],
            "counts": [int(c) for c in counts],
        },
    }


In [ ]:
"""
=== Semantic type inference ===
This cell defines `_semantic_type`, which labels each column as:
  boolean, integer, float, datetime, categorical, or text.
The function uses dtype checks + heuristics (e.g., 80% successful datetime parsing).
The output is more meaningful than raw pandas dtypes.
"""

import warnings

def _semantic_type(series: pd.Series) -> str:
    if pd.api.types.is_bool_dtype(series):
        return "boolean"
    if pd.api.types.is_numeric_dtype(series):
        return "integer" if pd.api.types.is_integer_dtype(series) else "float"
    if pd.api.types.is_datetime64_any_dtype(series):
        return "datetime"

    # try a light datetime guess on a small sample
    sample = series.dropna().astype(str).head(50)

    with warnings.catch_warnings():
        warnings.filterwarnings(
            "ignore",
            message="Could not infer format, so each element will be parsed individually.*",
            category=UserWarning,
        )
        parsed = pd.to_datetime(sample, errors="coerce")

    if parsed.notna().mean() >= 0.8:
        return "datetime"

    nunique = series.nunique(dropna=True)
    ratio = nunique / max(len(series), 1)
    return "categorical" if nunique <= 50 and ratio <= 0.5 else "text"


In [10]:
"""
=== Geospatial + Key Detection ===
This cell provides:
  - _is_lat / _is_lon: detect latitude/longitude columns
  - _geospatial_hints: determine whether dataset has lat/lon pair
  - _primary_key_candidates: detect columns that are unique + non-null
Used later to enrich dataset-level metadata.
"""

_LAT = {"lat", "latitude"}; _LON = {"lon", "lng", "long", "longitude"}

def _is_lat(name: str, s: pd.Series) -> bool:
    name = name.lower()
    if name in _LAT and pd.api.types.is_numeric_dtype(s): return True
    return pd.api.types.is_numeric_dtype(s) and s.min(skipna=True) >= -90 and s.max(skipna=True) <= 90

def _is_lon(name: str, s: pd.Series) -> bool:
    name = name.lower()
    if name in _LON and pd.api.types.is_numeric_dtype(s): return True
    return pd.api.types.is_numeric_dtype(s) and s.min(skipna=True) >= -180 and s.max(skipna=True) <= 180

def _geospatial_hints(df: pd.DataFrame) -> Dict[str, Any]:
    lat = next((c for c in df.columns if _is_lat(c, df[c])), None)
    lon = next((c for c in df.columns if _is_lon(c, df[c]) and c != lat), None)
    return {"lat_column": lat, "lon_column": lon, "has_latlon_pair": lat is not None and lon is not None}

def _primary_key_candidates(df: pd.DataFrame) -> List[str]:
    return [c for c in df.columns if df[c].isna().mean() == 0 and df[c].is_unique]

In [ ]:
"""
=== Full Data Profiler ===
This cell defines `profile_dataframe(df)` which:
  - Profiles each column (stats, missingness, semantic type, distributions, etc.)
  - Computes dataset-level metadata (row count, column count, pk candidates, geospatial hints)
  - Returns everything as a structured JSON-friendly dictionary.
This is the main profiling engine.
"""
def profile_dataframe(
    df: pd.DataFrame,
    *,
    sample_rows: int = 1_000_000,
    hist_bins: int = 10,
    topk_categorical: int = 10,
) -> Dict[str, Any]:
    if len(df) > sample_rows:
        df = df.sample(sample_rows, random_state=42)

    n_rows, n_cols = df.shape
    mem_bytes = int(df.memory_usage(deep=True).sum())

    column_summaries = {}
    type_counts: Dict[str, int] = {}

    for c in df.columns:
        s = df[c]
        sem = _semantic_type(s)
        type_counts[sem] = type_counts.get(sem, 0) + 1

        non_null = int(s.notna().sum())
        null_frac = float(s.isna().mean())
        nunique = int(s.nunique(dropna=True))
        is_unique = s.is_unique and null_frac == 0.0
        is_const = nunique == 1

        info: Dict[str, Any] = {
            "dtype": str(s.dtype),
            "semantic_type": sem,
            "non_null_count": non_null,
            "null_fraction": null_frac,
            "n_unique": nunique,
            "is_unique": bool(is_unique),
            "is_constant": bool(is_const),
            "example_values": [str(v) for v in s.dropna().unique()[:5]],
        }

        if sem in {"integer", "float"}:
            # Use JIT-accelerated numeric stats
            numeric_stats = numeric_stats_from_series(s, hist_bins=hist_bins)
            info.update(numeric_stats)

        elif sem == "datetime":
            s_dt, ok = _try_to_datetime(s)
            if ok and not s_dt.dropna().empty:
                info.update({
                    "min": s_dt.min(),
                    "max": s_dt.max(),
                    "is_monotonic_increasing": bool(s_dt.is_monotonic_increasing),
                    "is_monotonic_decreasing": bool(s_dt[::-1].is_monotonic_increasing),
                })

        elif sem == "categorical":
            info["top_values"] = _top_values(s.astype("string"), k=topk_categorical)

        elif sem == "text":
            info["text_stats"] = _text_stats(s)

        column_summaries[c] = info

    # dataset-level
    geohints = _geospatial_hints(df)
    pk = _primary_key_candidates(df)

    time_min, time_max = None, None
    for c, info in column_summaries.items():
        if info["semantic_type"] == "datetime" and info.get("min") is not None:
            tmin, tmax = info["min"], info["max"]
            time_min = tmin if time_min is None or (tmin is not None and tmin < time_min) else time_min
            time_max = tmax if time_max is None or (tmax is not None and tmax > time_max) else time_max

    return {
        "num_rows": int(n_rows),
        "num_columns": int(n_cols),
        "memory_usage_bytes": mem_bytes,
        "duplicate_row_fraction": float((n_rows - len(df.drop_duplicates())) / n_rows) if n_rows else 0.0,
        "type_counts": type_counts,
        "time_coverage": {"min": time_min, "max": time_max},
        "geospatial_hints": geohints,
        "primary_key_candidates": pk,
        "columns": column_summaries,
    }

# DATA SIGNAL

In [ ]:
import pandas as pd
import numpy as np
from typing import Dict
from typing import Tuple
from typing import Any
from typing import List

DOMAIN_KEYWORDS = {
    "cities_geo": [
        "city", "city_name", "latitude", "longitude",
        "lat", "lon", "country", "region", "state", "county"
    ],
    "health": [
        "patient", "hospital", "diagnosis", "disease",
        "blood_pressure", "cholesterol", "heart_rate",
        "bmi", "hospital_id"
    ],
    "finance": [
        "price", "revenue", "profit", "stock", "ticker",
        "transaction", "amount", "balance", "account_id"
    ],
    "demographics": [
        "age", "gender", "income", "education", "occupation",
        "household", "population"
    ],
}




def infer_domain_from_profile(profile: dict) -> str:
    """
    Heuristic: look at column names and pick the domain
    with the highest keyword overlap. Returns 'generic'
    if nothing matches.
    """
    columns = profile.get("columns", {})
    col_names = [name.lower() for name in columns.keys()]

    best_domain = "generic"
    best_score = 0

    for domain, keywords in DOMAIN_KEYWORDS.items():
        score = 0
        for col in col_names:
            for kw in keywords:
                if kw in col:
                    score += 1
        if score > best_score:
            best_score = score
            best_domain = domain

    if best_score == 0:
        return "generic"
    return best_domain


def build_data_signals(profile: Dict[str, Any]) -> Dict[str, Any]:
    """
    Turn the raw profiler output into higher-level 'data signals'
    that are easier for an LLM to consume.
    """
    columns = profile.get("columns", {})
    dataset = profile.get("dataset", {})

    # Handle both naming variants just in case
    n_rows = dataset.get("n_rows") or dataset.get("num_rows")
    n_cols = dataset.get("n_cols") or dataset.get("num_columns")

    # Dataset-level signals
    # Count semantic types
    type_counts: Dict[str, int] = {}
    total_missing = 0
    for name, info in columns.items():
        sem = info.get("semantic_type")
        if sem:
            type_counts[sem] = type_counts.get(sem, 0) + 1
        total_missing += int(info.get("n_missing", 0))

    total_cells = n_rows * n_cols if (n_rows is not None and n_cols is not None) else None
    overall_missing_fraction = (
        float(total_missing / total_cells) if total_cells and total_cells > 0 else None
    )

    # geospatial + primary keys may or may not exist depending on your profiler version
    geospatial = dataset.get("geospatial") or dataset.get("geospatial_hints") or {}
    pk_candidates = dataset.get("primary_keys") or dataset.get("primary_key_candidates") or []

    dataset_signals: Dict[str, Any] = {
        "n_rows": n_rows,
        "n_cols": n_cols,
        "type_counts": type_counts,                      # how many numeric / text / datetime / etc
        "overall_missing_fraction": overall_missing_fraction,
        "has_geospatial": bool(geospatial.get("has_latlon_pair")),
        "lat_column": geospatial.get("lat_column"),
        "lon_column": geospatial.get("lon_column"),
        "primary_key_candidates": pk_candidates,
    }

    # ---- Per-column signals ----
    column_signals: List[Dict[str, Any]] = []

    for name, info in columns.items():
        sem = info.get("semantic_type")
        n_missing = int(info.get("n_missing", 0))
        nunique = int(info.get("nunique") or info.get("n_unique", 0))

        missing_fraction = float(n_missing / n_rows) if n_rows and n_rows > 0 else 0.0

        # example values from top_values (if present)
        top_vals = info.get("top_values") or []
        example_values = [tv.get("value") for tv in top_vals[:3]] if isinstance(top_vals, list) else []

        # use 5% and 95% quantiles as an approximate range, if numeric
        quantiles = info.get("quantiles") or {}
        approx_min = quantiles.get("0.05")
        approx_max = quantiles.get("0.95")

        # simple heuristics for "id-like" and "binary" columns
        is_id_like = bool(n_rows and nunique == n_rows and missing_fraction == 0.0)
        is_binary = bool(
            nunique == 2
            and sem in {"integer", "float", "boolean", "categorical"}
        )

        column_signals.append(
            {
                "name": name,
                "semantic_type": sem,
                "missing_fraction": missing_fraction,
                "n_unique": nunique,
                "is_id_like": is_id_like,           # good candidate for primary key / identifier
                "is_binary": is_binary,             # good for yes/no flags
                "approx_min": approx_min,           # for numeric/datetime: rough range
                "approx_max": approx_max,
                "example_values": example_values,   # a few sample values as strings
            }
        )

    domain = infer_domain_from_profile(profile)

     #  Semantic tags / topic hints 
    tags = set()

    col_names = list(columns.keys())
    col_names_lower = [name.lower() for name in col_names]
    all_col_text = " ".join(col_names_lower)

    type_counts = dataset_signals.get("type_counts", {}) or {}

    # Geospatial-ish
    if any(tok in all_col_text for tok in ["lat", "lon", "latitude", "longitude", "city", "country", "region", "borough", "address"]):
        tags.add("geospatial")

    # Time / events
    if any(tok in all_col_text for tok in ["date", "time", "year", "month", "timestamp", "day"]):
        tags.add("temporal")

    # Finance / economics
    if any(tok in all_col_text for tok in ["price", "cost", "income", "revenue", "sales", "amount", "budget"]):
        tags.add("financial")

    # Demographics / people
    if any(tok in all_col_text for tok in ["age", "gender", "sex", "student", "employee", "customer", "user"]):
        tags.add("demographics")

    # Text-heavy datasets
    num_text_like = sum(
        count for t, count in type_counts.items()
        if str(t).lower() in {"text", "string", "categorical"}
    )
    if num_text_like and num_text_like >= len(type_counts) / 2:
        tags.add("text-heavy")

    # Metrics / numeric-heavy
    num_numeric_like = sum(
        count for t, count in type_counts.items()
        if any(kw in str(t).lower() for kw in ["int", "float", "number", "numeric"])
    )
    if num_numeric_like and num_numeric_like >= len(type_counts) / 2:
        tags.add("numeric-heavy")

    # Attach tags into dataset_signals and top-level signals
    dataset_signals["semantic_tags"] = sorted(tags)

    return {
        "dataset_signals": dataset_signals,
        "column_signals": column_signals,
        "domain": domain,
    }

# SUMMARY GENERATOR (LLM)

In [13]:

DOMAIN_PROMPT_HINTS = {
    "cities_geo": (
        "The dataset appears to be about cities and geographic indicators "
        "(e.g., air quality, population density, income, transport, happiness). "
        "Emphasize how columns relate to urban quality-of-life, cross-city comparisons, "
        "and geographic coverage."
    ),
    "health": (
        "The dataset appears to be about health / clinical data. "
        "Emphasize what each column measures clinically, potential privacy issues, "
        "and common analytical tasks like risk prediction or outcome analysis."
    ),
    "finance": (
        "The dataset appears to be about finance or transactions. "
        "Emphasize monetary units, time periods, and typical tasks like forecasting, "
        "risk analysis, and portfolio evaluation."
    ),
    "demographics": (
        "The dataset appears to be about demographics / population statistics. "
        "Emphasize population segments, sampling, and fairness / bias considerations."
    ),
    "generic": (
        "The dataset domain is not clearly identified. "
        "Give a general-purpose description suitable for data catalog documentation."
    ),
}


def summarize_dataset_with_llm(
    profile: Dict[str, Any],
    signals: Dict[str, Any],
    use_domain_hint: bool = True,
    model: str = openaimodel,
    side_info_text: Optional[str] = None,
) -> str:

    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    # ---- Convert dict → JSON ----
    dataset_summary_json = json.dumps(
        {
            "dataset_signals": signals.get("dataset_signals", {}),
            "column_signals": signals.get("column_signals", []),
        },
        indent=2,
        default=str
    )

    # Semantic tags (topic hints) from signals, if present
    semantic_tags = (
        signals.get("dataset_signals", {})
        .get("semantic_tags", [])
        or []
    )
    semantic_tags_text = ", ".join(semantic_tags) if semantic_tags else "none"

    data_signals = signals.get("dataset_signals", {}) or {}
    domain = data_signals.get("domain", "generic")
    domain_hint = ""


    if use_domain_hint:
        domain = signals.get("domain", "generic")
        domain_hint = DOMAIN_PROMPT_HINTS.get(domain, DOMAIN_PROMPT_HINTS["generic"])
    else:
        domain = "generic"
        domain_hint = (
            "Ignore any specific domain assumptions and write a generic dataset card "
            "without mentioning a particular application area."
        )

    side_info_section = ""
    if side_info_text:
        side_info_section = f"""
        Additional side information about this dataset (README or documentation snippet):

        {side_info_text}
        """
    # ---- Dataset-card prompt ----
    user_prompt = f"""
        You are a data documentation assistant.

        High-level domain: {domain!r}.
        Domain-specific guidance: {domain_hint}

        A machine-generated content summary of a tabular dataset in this JSON format:

        {dataset_summary_json}

        In addition, here are semantic topic tags inferred from the schema:
        {semantic_tags_text}

        The JSON has keys such as:
        - "dataset_signals" (n_rows, n_cols, type_counts, overall_missing_fraction, has_geospatial, etc.)
        - "column_signals": a list of columns with fields like name, semantic_type, missing_fraction, n_unique, example_values, etc.

        Your job is to write a VERY concise “dataset card” style description that:
        - Uses the JSON only to infer what the data contains and how it can be used.
        - Uses the user fields to tailor the framing to their interest.
        - Does NOT invent specific values (dates, row counts, monetary amounts, percentages) if they are not clearly implied by the JSON.

        =====================
        OUTPUT REQUIREMENTS
        =====================

        You MUST output in JSON format and follow this exact structure and HARD length caps:
        1) Section: "Columns"
           - A list of column names as key and types as value
        2) Section: "Key facts"\
           - Output 3–5 bullet points.
           - Each bullet MUST be ≤ 18 words.
           - Focus on: what each row roughly represents, main entities, key columns, important datatypes (dates, geo, text), and granularity.
           - If counts (rows, columns) are missing or null, speak qualitatively (e.g., "many records") instead of making them up.

        3) Section: "Use-cases"
           - Output 3–5 bullet points.
           - Each bullet MUST be ≤ 18 words.
           - Tailor to TOPIC and POTENTIAL_ANALYSES.
           - Include at least one analytical idea and one operational / business question.

        4) Section: "Caveats"
           - Output 3–5 bullet points.
           - Each bullet MUST be ≤ 18 words.
           - Mention schema limitations, missingness, lack of true coordinates, time coverage uncertainty, and any obvious biases from the columns.
           - If TIME_AND_GEO_COVERAGE is broader than what the schema suggests, flag that as a potential mismatch.

        5) Section: "Overview"
           - A single paragraph (NO bullets).
           - The paragraph MUST be between 40 and 200 words.
           - Add sentences that:
             * briefly state what each row represents, and
             * mention key columns by name (e.g., "School Year", "Borough", "Region", "Program").

        =====================
        STYLE & CONSTRAINTS
        =====================

        - Do NOT exceed any of the word caps, even if you must omit information.
        - Do NOT mention internal JSON structure, keys, or field names like "dataset_signals" or "column_signals".
        - Do NOT say "the JSON says" or "the schema indicates"; write as a normal human-facing description.
        - Do NOT fabricate: if something is unknown or ambiguous from the JSON, either omit it or state it as uncertain.
        - Be clear and non-technical; assume an analytically literate but non-expert reader.
        - Output ONLY the five sections in this order, with these exact headings:
        - Columns
        - Key facts
        - Use-cases
        - Caveats
        - Overview


        """

    response = client.responses.create(
        model=model,
        input=user_prompt,
    )

    return response.output_text


In [ ]:
def verify_column_names_and_types_deterministic(
    ground_truth_profile: Dict[str, Any],
    generated_summary_str: str
) -> Dict[str, Any]:
    """
    Deterministically verifies column names and their semantic types.
    """
    # Parse the generated_summary JSON string into a dictionary
    generated_summary_dict = json.loads(generated_summary_str)

    # Extract column names and types from the generated summary
    generated_columns = generated_summary_dict.get("Columns", {})

    # Extract column names and types from the ground truth profile
    ground_truth_columns = {
        name: info.get("semantic_type")
        for name, info in ground_truth_profile.get("columns", {}).items()
    }

    discrepancies = []

    # 1. Check for columns present in ground truth but missing in generated summary
    for gt_col_name, gt_sem_type in ground_truth_columns.items():
        # Standardize ground truth column name for comparison )
        standardized_gt_col_name = gt_col_name.replace('\n', ' ')

        found_in_generated = False
        for gen_col_name, gen_sem_type in generated_columns.items():
            if gen_col_name == gt_col_name or gen_col_name == standardized_gt_col_name:
                found_in_generated = True
                # Check semantic type consistency
                if gt_sem_type != gen_sem_type:
                    discrepancies.append(
                        f"Column '{gt_col_name}': Ground truth type '{gt_sem_type}' "
                        f"does not match generated type '{gen_sem_type}'."
                    )
                break

        if not found_in_generated:
            discrepancies.append(
                f"Column '{gt_col_name}' (type: {gt_sem_type}) is in ground truth but missing from generated summary."
            )

    # 2. Check for columns present in generated summary but missing in ground truth
    for gen_col_name, gen_sem_type in generated_columns.items():
        found_in_ground_truth = False
        for gt_col_name in ground_truth_columns.keys():
            standardized_gt_col_name = gt_col_name.replace('\n', ' ')
            if gen_col_name == gt_col_name or gen_col_name == standardized_gt_col_name:
                found_in_ground_truth = True
                break
        if not found_in_ground_truth:
            discrepancies.append(
                f"Column '{gen_col_name}' (type: {gen_sem_type}) is in generated summary but missing from ground truth."
            )

    return {
        "column_type_discrepancies": discrepancies,
        "status": "Passed" if not discrepancies else "Failed"
    }


### Evaluation Section

This section evaluates the generated summary against a ground truth summary using ROUGE-L and BERTScore metrics.

In [15]:
import json
from typing import Union, Dict, Any

def parse_llm_summary(raw: Union[str, Dict[str, Any]]) -> Dict[str, Any]:
    if isinstance(raw, dict):
        return raw

    if raw is None:
        raise ValueError("summary is None; did the LLM call fail?")

    text = str(raw).strip()

    # Strip ```json ... ``` or ``` ... ``` fences if ever present
    if text.startswith("```"):
        lines = text.splitlines()
        if lines and lines[0].startswith("```"):
            lines = lines[1:]
        if lines and lines[-1].startswith("```"):
            lines = lines[:-1]
        text = "\n".join(lines).strip()

    start = text.find("{")
    end = text.rfind("}")
    if start != -1 and end != -1:
        text = text[start:end+1]

    return json.loads(text)


In [16]:
from typing import Dict, Any, Optional, Union

def extract_overview_from_summary(summary_json: Union[Dict[str, Any], str, None]) -> str:
    """Safely extract the 'Overview' field from a generated or existing summary JSON."""
    if summary_json is None:
        return ""

    if isinstance(summary_json, str):
        try:
            summary_json = parse_llm_summary(summary_json)
        except Exception:
            return str(summary_json).strip()

    if isinstance(summary_json, dict):
        return str(summary_json.get("Overview", "")).strip()

    return str(summary_json).strip()


def evaluate_generated_overview_vs_reference(
    generated_summary_json: Dict[str, Any],
    reference_overview_text: str,
    reference_label: str = "Reference Overview",
) -> Dict[str, Any]:
    """Compute ROUGE-L and BERTScore between generated Overview and a reference Overview.

    Returns a dict with metric values, and also prints a human-readable report.
    """
    from rouge_score import rouge_scorer
    from evaluate import load

    generated_overview = extract_overview_from_summary(generated_summary_json)
    reference_overview_text = (reference_overview_text or "").strip()

    print("Generated Summary Overview:\n", generated_overview, "\n")
    print(f"{reference_label}:\n", reference_overview_text, "\n")

    # Guard
    if not reference_overview_text:
        print("No reference Overview provided; skipping ROUGE/BERTScore.")
        return {
            "skipped": True,
            "reason": "empty_reference",
            "rougeL": None,
            "bertscore": None,
        }

    # ROUGE-L
    scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)
    rouge_scores = scorer.score(reference_overview_text, generated_overview)
    rougeL = rouge_scores["rougeL"]

    print("ROUGE-L Results:")
    print(f"  Precision: {rougeL.precision:.4f}")
    print(f"  Recall:    {rougeL.recall:.4f}")
    print(f"  F1:        {rougeL.fmeasure:.4f}")

    # BERTScore
    bertscore = load("bertscore")
    results = bertscore.compute(
        predictions=[generated_overview],
        references=[reference_overview_text],
        lang="en",
    )

    bert_dict = {
        "precision": float(results["precision"][0]),
        "recall": float(results["recall"][0]),
        "f1": float(results["f1"][0]),
    }

    print("\nBERTScore Results:")
    print(f"  Precision: {bert_dict['precision']:.4f}")
    print(f"  Recall:    {bert_dict['recall']:.4f}")
    print(f"  F1:        {bert_dict['f1']:.4f}")

    return {
        "skipped": False,
        "rougeL": {
            "precision": float(rougeL.precision),
            "recall": float(rougeL.recall),
            "f1": float(rougeL.fmeasure),
        },
        "bertscore": bert_dict,
    }


def evaluate_summary_vs_baseline(
    generated_summary_json: Dict[str, Any],
    baseline_text: str,
) -> Dict[str, Any]:
    """Backward-compatible wrapper for the earlier notebook flow."""
    return evaluate_generated_overview_vs_reference(
        generated_summary_json=generated_summary_json,
        reference_overview_text=baseline_text,
        reference_label="Ground-truth-based Baseline Summary",
    )


# Example (safe-guarded) usage:
if "summary" in globals() and "baseline_summary" in globals():
    try:
        generated_summary_dict = parse_llm_summary(summary)
        evaluate_summary_vs_baseline(generated_summary_dict, baseline_summary)
    except Exception as e:
        print(f"Evaluation example skipped due to error: {e}")


In [ ]:
def verify_summary_with_llm(
    ground_truth_profile: Dict[str, Any],
    generated_summary_str: str,
    model: str = openaimodel
) -> str:

    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    # This preprocessing step corrects common malformations. We will perform these fixes here because they are common patterns in the LLM's output for this specific task.
    cleaned_summary_str = generated_summary_str
    cleaned_summary_str = cleaned_summary_str.replace('},\n  "Use-cases":', '],\n  "Use-cases":')
    cleaned_summary_str = cleaned_summary_str.replace('},\n  "Caveats":', '],\n  "Caveats":')
    cleaned_summary_str = cleaned_summary_str.replace('},\n  "Overview":', '],\n  "Overview":')

    # Parse the cleaned_summary_str JSON string into a dictionary
    generated_summary_dict = json.loads(cleaned_summary_str)

    verification_input = {
        "ground_truth_profile": {
            "num_rows": ground_truth_profile.get("num_rows"),
            "num_columns": ground_truth_profile.get("num_columns"),
            "type_counts": ground_truth_profile.get("type_counts"),
            "duplicate_row_fraction": ground_truth_profile.get("duplicate_row_fraction"),
            "time_coverage": ground_truth_profile.get("time_coverage"),
            "geospatial_hints": ground_truth_profile.get("geospatial_hints"),
            "primary_key_candidates": ground_truth_profile.get("primary_key_candidates"),
            "column_details": {
                col_name: {
                    "semantic_type": col_info.get("semantic_type"),
                    "n_unique": col_info.get("n_unique"),
                    "null_fraction": col_info.get("null_fraction"),
                    "min": col_info.get("min"),
                    "max": col_info.get("max"),
                    "example_values": col_info.get("example_values", [])
                } for col_name, col_info in ground_truth_profile.get("columns", {}).items()
            }
        },
        "generated_summary": generated_summary_dict
    }

    verification_json_input = json.dumps(verification_input, indent=2, default=str)

    user_prompt = f"""
        You are an AI assistant designed to verify the factual consistency of a dataset description against ground truth data.

        Here is a JSON object containing `ground_truth_profile` (from a data profiler) and a `generated_summary` (previously created by another LLM).

        {verification_json_input}

        Your task is to:
        1. **Key Facts Verification**: Evaluate the accuracy and consistency of the "Key facts" section in `generated_summary` against the `ground_truth_profile` (e.g., num_rows, num_columns, geospatial hints, primary key candidates).
        2. **Caveats Verification**: Assess if the "Caveats" section accurately reflects potential data limitations or issues present in the `ground_truth_profile` (e.g., null fractions, time coverage, geospatial info).
        3. **Overall Assessment**: Provide a concluding statement on the general accuracy and consistency of the `generated_summary` with respect to the `ground_truth_profile`.

        OUTPUT REQUIREMENTS:
        Output your verification report in JSON format with the following keys:
        - "Result": A string indicating the overall verification status, either "Passed" or "Failed". This should be determined by whether the summary is generally accurate and consistent, with no factual contradictions.
        - "Key Facts Consistency": A string comment on the accuracy of the "Key facts" section.
        - "Caveats Accuracy": A string comment on the accuracy of the "Caveats" section.
        - "Overall Verification Status": A concluding statement on the summary's accuracy.
        """

    response = client.responses.create(
        model=model,
        input=user_prompt,
    )

    return response.output_text

Batch Run Over S3 Datasets

In [ ]:
# === Batch: run dataset description pipeline over S3 ===

from datetime import datetime, timezone
import os
import io
import gc
import json
import inspect
from functools import lru_cache
from typing import List, Optional, Dict, Any, Iterable, Tuple

import pandas as pd


# -----------------------
# Config
# -----------------------
BUCKET_NAME = "nyu-tandon-cs-gy-6513-project"

AUCTUS_NO_DESC_PREFIX = "auctus/dataset_no_description/"
AUCTUS_HAS_DESC_PREFIX = "auctus/dataset_has_description/"

NYC_OPEN_DATA_PREFIX = "nyc_open_data/"
NYC_DATA_CATALOG_KEY = "nyc_open_data/dataset_catalog.csv"

OUTPUT_PREFIX = "dataset_descriptions/"
FAIL_PREFIX = "dataset_descriptions_failures/"

# ---- RAM safety knobs ----
SAMPLE_ROWS = 3000
MAX_COLS = 200
SKIP_IF_FILE_MB = 300
PRINT_EVERY = 25

# ---- Retry + validation knobs ----
MAX_RETRIES = 2  # total attempts = 1 + MAX_RETRIES
USE_LLM_JUDGE = True  


# -----------------------
# Utilities
# -----------------------
def utc_now_iso() -> str:
    return datetime.now(timezone.utc).isoformat()


def _list_keys(bucket: str, prefix: str) -> List[str]:
    keys = []
    token = None
    while True:
        kwargs = {"Bucket": bucket, "Prefix": prefix}
        if token:
            kwargs["ContinuationToken"] = token
        resp = s3.list_objects_v2(**kwargs)
        for obj in resp.get("Contents", []):
            keys.append(obj["Key"])
        if resp.get("IsTruncated"):
            token = resp.get("NextContinuationToken")
        else:
            break
    return keys


def iter_dataset_keys(
    bucket: str,
    prefixes: List[str],
    suffix: str = ".csv",
) -> Iterable[str]:
    for prefix in prefixes:
        token = None
        while True:
            kwargs = {"Bucket": bucket, "Prefix": prefix}
            if token:
                kwargs["ContinuationToken"] = token
            resp = s3.list_objects_v2(**kwargs)

            for obj in resp.get("Contents", []):
                key = obj["Key"]
                if not key.lower().endswith(suffix):
                    continue
                if key == NYC_DATA_CATALOG_KEY:
                    continue
                yield key

            if resp.get("IsTruncated"):
                token = resp.get("NextContinuationToken")
            else:
                break


def _safe_output_name(dataset_key: str) -> str:
    no_ext = os.path.splitext(dataset_key)[0]
    return no_ext.replace("/", "__")


def _replace_ext(key: str, new_ext: str) -> str:
    base, _ = os.path.splitext(key)
    return base + new_ext


def _head_size_mb(bucket: str, key: str) -> float:
    try:
        head = s3.head_object(Bucket=bucket, Key=key)
        return head.get("ContentLength", 0) / (1024 * 1024)
    except Exception:
        return 0.0


def _read_csv_sample_from_s3(
    bucket: str,
    key: str,
    sample_rows: int = SAMPLE_ROWS,
    max_cols: int = MAX_COLS,
) -> pd.DataFrame:
    obj = s3.get_object(Bucket=bucket, Key=key)
    header_df = pd.read_csv(obj["Body"], nrows=0)
    cols = list(header_df.columns)[:max_cols]

    obj = s3.get_object(Bucket=bucket, Key=key)
    df = pd.read_csv(
        obj["Body"],
        nrows=sample_rows,
        usecols=cols,
        low_memory=True
    )
    return df


def _cleanup(*vars_):
    for v in vars_:
        try:
            del v
        except Exception:
            pass
    gc.collect()


def _extract_overview_from_any_json(d: Dict[str, Any]) -> Optional[str]:
    for k in ["Overview", "overview", "summary", "Summary", "description", "Description"]:
        v = d.get(k)
        if isinstance(v, str) and v.strip():
            return v.strip()

    if isinstance(d.get("generated"), dict):
        for k in ["Overview", "overview", "summary", "description"]:
            v = d["generated"].get(k)
            if isinstance(v, str) and v.strip():
                return v.strip()
    return None


# -----------------------
# NYC catalog loader (3-col schema)
# -----------------------
@lru_cache(maxsize=1)
def _load_nyc_catalog_map(bucket: str = BUCKET_NAME) -> Dict[str, str]:
    try:
        obj = s3.get_object(Bucket=bucket, Key=NYC_DATA_CATALOG_KEY)
        body = obj["Body"].read()
        df = pd.read_csv(io.BytesIO(body), header=None, low_memory=True)

        if df.shape[1] < 3:
            print(f"Warning: NYC catalog has {df.shape[1]} columns; expected >= 3.")
            return {}

        mp = {}
        for _, row in df.iterrows():
            name = str(row[1]).strip()
            summary = str(row[2]).strip()

            if not name or not summary:
                continue
            if not name.lower().endswith(".csv"):
                name = name + ".csv"

            mp[name.lower()] = summary

        return mp
    except Exception as e:
        print(f"Could not load NYC Open Data catalog ({NYC_DATA_CATALOG_KEY}): {e}")
        return {}


def load_existing_overview_text(
    dataset_key: str,
    bucket: str = BUCKET_NAME,
) -> Optional[str]:

    if dataset_key.startswith(AUCTUS_HAS_DESC_PREFIX):
        json_key = _replace_ext(dataset_key, ".json")
        try:
            obj = s3.get_object(Bucket=bucket, Key=json_key)
            raw = obj["Body"].read().decode("utf-8", errors="ignore")

            try:
                existing_json = parse_llm_summary(raw)
                ov = extract_overview_from_summary(existing_json)
                if ov:
                    return ov
            except Exception:
                pass

            existing_dict = json.loads(raw)
            return _extract_overview_from_any_json(existing_dict)

        except Exception as e:
            print(f"Warning: could not load existing Auctus description for {dataset_key}: {e}")
            return None

    if dataset_key.startswith(NYC_OPEN_DATA_PREFIX):
        catalog_map = _load_nyc_catalog_map(bucket)
        fname = os.path.basename(dataset_key).lower()
        return catalog_map.get(fname)

    return None


# -----------------------
# Rule-based validation
# -----------------------
REQUIRED_FIELDS = ["Columns", "Key facts", "Use-cases", "Caveats", "Overview"]

def _normalize_type(t: str) -> str:
    t = str(t or "").lower().strip()
    # collapse common synonyms
    if t in {"int", "integer", "long"}:
        return "integer"
    if t in {"float", "double", "decimal", "numeric", "number"}:
        return "numeric"
    if "date" in t or "time" in t:
        return "datetime"
    if t in {"bool", "boolean"}:
        return "boolean"
    if t in {"cat", "category", "categorical"}:
        return "categorical"
    if t in {"text", "string", "str"}:
        return "text"
    return t

def _profile_columns_map(profile: Dict[str, Any]) -> Dict[str, Dict[str, Any]]:
    """
    Try to locate the column-level map in your profiler output.
    """
    if not isinstance(profile, dict):
        return {}
    cols = profile.get("columns")
    if isinstance(cols, dict):
        return cols
    # fallback: sometimes nested
    for k in ["column_profiles", "schema", "fields"]:
        v = profile.get(k)
        if isinstance(v, dict):
            return v
    return {}

def _profile_type_for(col_profile: Dict[str, Any]) -> str:
    for k in ["semantic_type", "type", "inferred_type", "dtype"]:
        v = col_profile.get(k)
        if isinstance(v, str) and v.strip():
            return _normalize_type(v)
    return ""

def _extract_generated_columns(gen_json: Dict[str, Any]) -> List[Tuple[str, str]]:
    """
    Returns list of (name, type) from the LLM JSON.
    Supports:
      - list of dicts: [{"name":..., "type":...}, ...]
      - list of strings: ["colA (numeric)", ...]  (best-effort)
      - dict mapping: {"colA":"numeric", ...}
    """
    cols = gen_json.get("Columns")
    out = []

    if isinstance(cols, dict):
        for name, t in cols.items():
            out.append((str(name).strip(), _normalize_type(t)))
        return out

    if isinstance(cols, list):
        for item in cols:
            if isinstance(item, dict):
                name = str(item.get("name") or item.get("column") or "").strip()
                t = item.get("type") or item.get("semantic_type") or ""
                if name:
                    out.append((name, _normalize_type(t)))
            elif isinstance(item, str):
                # best-effort parse "name (type)"
                s = item.strip()
                if "(" in s and s.endswith(")"):
                    name = s.split("(")[0].strip()
                    t = s.split("(")[-1].rstrip(")").strip()
                    out.append((name, _normalize_type(t)))
                else:
                    out.append((s, ""))
    return out

def rule_based_validate_summary(
    summary_text: str,
    profile: Dict[str, Any]
) -> Tuple[bool, List[str], Optional[Dict[str, Any]]]:
    issues = []

    # 1) Parse JSON
    try:
        gen_json = parse_llm_summary(summary_text) if not isinstance(summary_text, dict) else summary_text
    except Exception:
        return False, ["Output is not valid JSON or does not match expected JSON format."], None

    if not isinstance(gen_json, dict):
        return False, ["Parsed output is not a JSON object."], None

    # 2) Required fields
    for f in REQUIRED_FIELDS:
        if f not in gen_json:
            issues.append(f"Missing required field: {f}")

    # 3) Overview length sanity (soft)
    ov = gen_json.get("Overview")
    if isinstance(ov, str):
        wc = len(ov.split())
        if wc < 25 or wc > 120:
            issues.append("Overview length seems outside the intended concise range.")

    # 4) Column name and type consistency
    prof_cols = _profile_columns_map(profile)
    prof_col_names = {c.strip() for c in prof_cols.keys()}

    gen_cols = _extract_generated_columns(gen_json)
    if gen_cols:
        # name existence checks
        for name, _t in gen_cols:
            if name and prof_col_names and name not in prof_col_names:
                issues.append(f"Generated column not found in schema: {name}")

        # type checks (best-effort, tolerant)
        for name, t in gen_cols:
            if not name or not t:
                continue
            col_prof = prof_cols.get(name)
            if isinstance(col_prof, dict):
                ptype = _profile_type_for(col_prof)
                if ptype and t:
                    # allow integer vs numeric compatibility
                    if not (
                        (ptype == t) or
                        (ptype in {"integer", "numeric"} and t in {"integer", "numeric"})
                    ):
                        issues.append(f"Type mismatch for {name}: generated={t}, profiler={ptype}")

    ok = len(issues) == 0
    return ok, issues, gen_json


def llm_judge_pass(
    profile: Dict[str, Any],
    signals: Dict[str, Any],
    generated_json: Dict[str, Any]
) -> Tuple[bool, List[str]]:
    """
    Call the verify_summary_with_llm() judge function to evaluate the candidate dataset card.

    Expected verify_summary_with_llm return format:
        {
          "pass": bool,
          "issues": ["issue 1", "issue 2", ...]
        }
    """
    if not USE_LLM_JUDGE:
        return True, []

    try:
        # verify_summary_with_llm should be defined in the same file
        out = verify_summary_with_llm(profile, generated_json)

        if isinstance(out, dict):
            passed = bool(out.get("pass", True))
            issues = out.get("issues") or []
            # Ensure issues are strings
            issues = [str(i) for i in issues]
            return passed, issues

        # If it returns something unexpected, don't block the pipeline
        return True, []

    except Exception as e:
        # Fail-open: do not break the main pipeline if judge fails
        print(f"LLM judge (verify_summary_with_llm) failed: {e}")
        return True, []



# -----------------------
# Safe call wrapper for summarize_dataset_with_llm
# -----------------------
def _call_summarizer(profile, signals, feedback: Optional[str] = None):
    fn = summarize_dataset_with_llm
    if feedback:
        try:
            sig = inspect.signature(fn)
            for param in ["feedback", "repair_instructions", "additional_instructions", "extra_instructions"]:
                if param in sig.parameters:
                    return fn(profile, signals, **{param: feedback})
        except Exception:
            pass
    return fn(profile, signals)


def generate_with_retry(
    profile: Dict[str, Any],
    signals: Dict[str, Any],
    max_retries: int = MAX_RETRIES
) -> Tuple[Optional[str], Optional[Dict[str, Any]], List[Dict[str, Any]]]:
    """
    Returns:
      summary_text, parsed_json, attempts_log
    """
    attempts = []
    feedback = None

    for attempt_idx in range(max_retries + 1):
        summary_text = _call_summarizer(profile, signals, feedback=feedback)

        rb_ok, rb_issues, parsed = rule_based_validate_summary(summary_text, profile)
        judge_ok, judge_issues = llm_judge_pass(profile, signals, parsed or {})

        ok = rb_ok and judge_ok

        attempts.append({
            "attempt": attempt_idx + 1,
            "rule_based_ok": rb_ok,
            "rule_based_issues": rb_issues,
            "llm_judge_ok": judge_ok,
            "llm_judge_issues": judge_issues,
        })

        if ok:
            return summary_text, parsed, attempts

        # Build corrective feedback for next try
        all_issues = rb_issues + judge_issues
        feedback = (
            "Fix the following issues and regenerate the SAME JSON structure:\n"
            + "\n".join([f"- {i}" for i in all_issues])
            + "\nDo not add columns not in the schema. Ensure required fields exist."
        )

    return None, None, attempts


# -----------------------
# Core runner
# -----------------------
def _run_single_dataset(
    key: str,
    bucket_name: str,
    run_type: str,
    output_prefix: str,
    compute_eval: bool
) -> bool:

    size_mb = _head_size_mb(bucket_name, key)
    if size_mb and size_mb > SKIP_IF_FILE_MB:
        print(f"[SKIP huge file] {key} ({size_mb:.1f} MB)")
        return False

    try:
        df = _read_csv_sample_from_s3(bucket_name, key)

        if df.shape[1] > MAX_COLS:
            df = df.iloc[:, :MAX_COLS]

        existing_overview = None
        if run_type == "has_existing_description":
            existing_overview = load_existing_overview_text(key, bucket=bucket_name)

        profile = profile_dataframe(df)
        signals = build_data_signals(profile)

        # --- NEW: generation with retry + validation ---
        summary_text, summary_json, attempts_log = generate_with_retry(profile, signals)

        if not summary_text or not summary_json:
            # Save failure artifact and skip normal persistence
            safe_name = _safe_output_name(key)
            fail_key = f"{FAIL_PREFIX}{safe_name}__failed.json"

            fail_obj = {
                "run_type": run_type,
                "dataset_key": key,
                "existing_overview": existing_overview,
                "profile": profile,
                "signals": signals,
                "summary": None,
                "validation_attempts": attempts_log,
                "processed_at_utc": utc_now_iso(),
                "sample_rows_used": SAMPLE_ROWS,
                "max_cols_used": MAX_COLS,
                "source_file_size_mb": size_mb,
                "status": "failed_after_retries",
            }

            s3.put_object(
                Bucket=bucket_name,
                Key=fail_key,
                Body=json.dumps(fail_obj, indent=2, default=str).encode("utf-8"),
                ContentType="application/json",
            )

            print(f"[FAILED] Saved failure artifact to s3://{bucket_name}/{fail_key}")

            _cleanup(df, profile, signals, fail_obj)
            return False

        # Normal success result
        result = {
            "run_type": run_type,
            "dataset_key": key,
            "existing_overview": existing_overview,
            "profile": profile,
            "signals": signals,
            "summary": summary_text,
            "parsed_summary": summary_json,  # helpful for debugging/reuse
            "validation_attempts": attempts_log,
            "eval_vs_existing": None,  # still computed in a post-run cell
            "processed_at_utc": utc_now_iso(),
            "sample_rows_used": SAMPLE_ROWS,
            "max_cols_used": MAX_COLS,
            "source_file_size_mb": size_mb,
            "status": "success",
        }

        safe_name = _safe_output_name(key)
        output_key = f"{output_prefix}{safe_name}__description.json"

        s3.put_object(
            Bucket=bucket_name,
            Key=output_key,
            Body=json.dumps(result, indent=2, default=str).encode("utf-8"),
            ContentType="application/json",
        )

        print(f"Saved summary to s3://{bucket_name}/{output_key}")

        _cleanup(df, profile, signals, result)
        return True

    except Exception as e:
        print(f"ERROR processing {key}: {e}")
        _cleanup()
        return False


def run_pipeline_no_existing_descriptions(
    bucket_name: str = BUCKET_NAME,
    prefixes: Optional[List[str]] = None,
    output_prefix: str = OUTPUT_PREFIX,
    max_datasets: Optional[int] = None,
) -> Dict[str, int]:

    if prefixes is None:
        prefixes = [AUCTUS_NO_DESC_PREFIX]

    processed = 0
    failed = 0
    seen = 0

    for key in iter_dataset_keys(bucket_name, prefixes=prefixes, suffix=".csv"):
        if max_datasets is not None and processed >= max_datasets:
            break

        seen += 1
        print(f"\n=== [NO DESC] Processing {key} ===")

        ok = _run_single_dataset(
            key=key,
            bucket_name=bucket_name,
            run_type="no_existing_description",
            output_prefix=output_prefix,
            compute_eval=False
        )

        if ok:
            processed += 1
        else:
            failed += 1

        if seen % PRINT_EVERY == 0:
            print(f"[NO DESC] Progress: processed={processed}, failed={failed}, seen={seen}")

    print(f"\n[NO DESC] Done. Processed: {processed}, Failed: {failed}")
    return {"processed": processed, "failed": failed}


def run_pipeline_with_existing_descriptions(
    bucket_name: str = BUCKET_NAME,
    prefixes: Optional[List[str]] = None,
    output_prefix: str = OUTPUT_PREFIX,
    max_datasets: Optional[int] = None,
) -> Dict[str, int]:

    if prefixes is None:
        prefixes = [AUCTUS_HAS_DESC_PREFIX, NYC_OPEN_DATA_PREFIX]

    processed = 0
    failed = 0
    seen = 0

    for key in iter_dataset_keys(bucket_name, prefixes=prefixes, suffix=".csv"):
        if max_datasets is not None and processed >= max_datasets:
            break

        seen += 1
        print(f"\n=== [HAS DESC] Processing {key} ===")

        ok = _run_single_dataset(
            key=key,
            bucket_name=bucket_name,
            run_type="has_existing_description",
            output_prefix=output_prefix,
            compute_eval=False
        )

        if ok:
            processed += 1
        else:
            failed += 1

        if seen % PRINT_EVERY == 0:
            print(f"[HAS DESC] Progress: processed={processed}, failed={failed}, seen={seen}")

    print(f"\n[HAS DESC] Done. Processed: {processed}, Failed: {failed}")
    return {"processed": processed, "failed": failed}


# -----------------------
# 🔹 Batch runs
# -----------------------
# Set max_datasets=None to run all.
# Using 0 means "run none" (kept for safety).

run_pipeline_no_existing_descriptions(max_datasets=0)
run_pipeline_with_existing_descriptions(max_datasets=0)



[NO DESC] Done. Processed: 0, Failed: 0

[HAS DESC] Done. Processed: 0, Failed: 0


{'processed': 0, 'failed': 0}

# Search impact evaluation (offline retrieval)

This  section measures whether adding our generated descriptions improves
top-*k* retrieval performance on a held-out query set.
 

1. Build two searchable corpora:
   - **Baseline**: existing descriptions (when available).
   - **Augmented**: our generated descriptions (Overview).

2. Embed each corpus and run vector retrieval.

3. Compute **precision@k** and **recall@k** against a  query set with
   manually curated relevant dataset keys.

In [ ]:
# --- Generate an UNBIASED synthetic query_set.json for offline retrieval evaluation ---
# This version avoids using baseline or generated descriptions.
# Queries are derived ONLY from:
#   - dataset filename
#   - column headers (schema)
#
# This makes the query set fair for BEFORE vs AFTER description comparisons.

import json
import re
import os
from pathlib import Path
from typing import List, Dict, Tuple, Optional

import boto3
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer


# -----------------------
# Config
# -----------------------
BUCKET = "nyu-tandon-cs-gy-6513-project"

AUCTUS_NO_DESC_PREFIX = "auctus/dataset_no_description/"
AUCTUS_HAS_DESC_PREFIX = "auctus/dataset_has_description/"
NYC_PREFIX = "nyc_open_data/"

PREFIXES = [AUCTUS_NO_DESC_PREFIX, AUCTUS_HAS_DESC_PREFIX, NYC_PREFIX]

LOCAL_OUT_PATH = Path("evaluation/query_set.json")
S3_OUT_KEY = "evaluation/query_set.json"

# How many datasets to sample for query generation
MAX_DATASETS = 1500  

# Queries per dataset
QUERIES_PER_DATASET = 1

# TF-IDF settings over schema text
NGRAM_RANGE = (1, 2)
MAX_FEATURES = 8000
STOP_WORDS = "english"

RANDOM_SEED = 42

# Safety: ignore the NYC catalog file as a dataset
NYC_CATALOG_BASENAMES = {"dataset_catalog.csv", "data_catalog.csv", "dataset catalog.csv"}


# -----------------------
# Helpers
# -----------------------
def _get_s3_client():
    return boto3.client("s3")


def _list_s3_keys(s3, bucket: str, prefix: str) -> List[str]:
    keys = []
    token = None
    while True:
        kwargs = {"Bucket": bucket, "Prefix": prefix}
        if token:
            kwargs["ContinuationToken"] = token
        resp = s3.list_objects_v2(**kwargs)
        for obj in resp.get("Contents", []):
            keys.append(obj["Key"])
        if resp.get("IsTruncated"):
            token = resp.get("NextContinuationToken")
        else:
            break
    return keys


def iter_dataset_csv_keys(s3, bucket: str, prefixes: List[str]) -> List[str]:
    out = []
    for p in prefixes:
        keys = _list_s3_keys(s3, bucket, p)
        for k in keys:
            if not k.lower().endswith(".csv"):
                continue
            base = os.path.basename(k).lower()
            if base in NYC_CATALOG_BASENAMES:
                continue
            out.append(k)
    # dedupe while preserving order
    seen = set()
    deduped = []
    for k in out:
        if k not in seen:
            seen.add(k)
            deduped.append(k)
    return deduped


def _clean_text(t: str) -> str:
    t = re.sub(r"[_\-]+", " ", str(t))
    t = re.sub(r"\s+", " ", t)
    return t.strip()


def _read_csv_header_columns(s3, bucket: str, key: str) -> List[str]:
    obj = s3.get_object(Bucket=bucket, Key=key)
    df0 = pd.read_csv(obj["Body"], nrows=0)
    cols = [str(c) for c in df0.columns]
    return cols


def build_schema_text(dataset_key: str, columns: List[str]) -> str:
    """
    A neutral text representation of a dataset intended for query synthesis.
    """
    fname = os.path.splitext(os.path.basename(dataset_key))[0]
    parts = [fname] + columns
    return _clean_text(" ".join(parts))


def _keywords_tfidf(texts: List[str], top_k: int = 6) -> List[List[str]]:
    vectorizer = TfidfVectorizer(
        stop_words=STOP_WORDS,
        ngram_range=NGRAM_RANGE,
        max_features=MAX_FEATURES
    )
    X = vectorizer.fit_transform(texts)
    vocab = vectorizer.get_feature_names_out()

    keywords_per_doc = []
    for i in range(X.shape[0]):
        row = X.getrow(i)
        if row.nnz == 0:
            keywords_per_doc.append([])
            continue

        sorted_idx = row.data.argsort()[::-1]
        top_indices = row.indices[sorted_idx][:top_k]
        kws = [vocab[j] for j in top_indices]
        keywords_per_doc.append(kws)

    return keywords_per_doc


def _build_query_from_keywords(kws: List[str], variant: int = 0) -> Optional[str]:
    if not kws:
        return None
    if variant == 0:
        return " ".join(kws[:3])
    if variant == 1:
        return " ".join(kws[:2])
    return kws[0]


def generate_unbiased_query_set(
    dataset_keys: List[str],
    max_datasets: int = MAX_DATASETS,
    queries_per_dataset: int = QUERIES_PER_DATASET
) -> List[Dict]:

    import random
    random.seed(RANDOM_SEED)

    # shuffle and cap
    keys = dataset_keys[:]
    random.shuffle(keys)
    keys = keys[:max_datasets]

    schema_texts = []
    kept_keys = []

    for k in keys:
        try:
            cols = _read_csv_header_columns(s3, BUCKET, k)
            txt = build_schema_text(k, cols)
            if txt:
                schema_texts.append(txt)
                kept_keys.append(k)
        except Exception:
            continue

    print(f"Built schema texts for {len(kept_keys)} datasets.")

    kws_list = _keywords_tfidf(schema_texts, top_k=8)

    query_set = []
    for dataset_key, kws, schema_txt in zip(kept_keys, kws_list, schema_texts):
        for q_idx in range(queries_per_dataset):
            q = _build_query_from_keywords(kws, variant=q_idx)
            if not q:
                # fallback: short chunk of schema text
                q = " ".join(schema_txt.split()[:5])

            q = _clean_text(q)

            query_set.append({
                "query": q,
                "relevant_datasets": [dataset_key],
                "source_dataset": dataset_key,
                "generation_method": "schema+filename_tfidf"
            })

    return query_set


def save_query_set_local(query_set: List[Dict], path: Path):
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open("w", encoding="utf-8") as f:
        json.dump(query_set, f, indent=2, ensure_ascii=False)
    print(f"Saved {len(query_set)} queries to {path}")


def upload_query_set_to_s3(s3, bucket: str, key: str, path: Path):
    s3.upload_file(str(path), bucket, key)
    print(f"Uploaded query set to s3://{bucket}/{key}")


# -----------------------
# Execute
# -----------------------
s3 = _get_s3_client()

all_dataset_keys = iter_dataset_csv_keys(s3, BUCKET, PREFIXES)
print(f"Found {len(all_dataset_keys)} dataset CSV keys across prefixes.")

query_set = generate_unbiased_query_set(
    all_dataset_keys,
    max_datasets=MAX_DATASETS,
    queries_per_dataset=QUERIES_PER_DATASET
)

save_query_set_local(query_set, LOCAL_OUT_PATH)

try:
    upload_query_set_to_s3(s3, BUCKET, S3_OUT_KEY, LOCAL_OUT_PATH)
except Exception as e:
    print("S3 upload skipped/failed:", repr(e))

query_set[:5]


Found 1339 dataset CSV keys across prefixes.
Built schema texts for 1339 datasets.
Saved 1339 queries to evaluation/query_set.json
Uploaded query set to s3://nyu-tandon-cs-gy-6513-project/evaluation/query_set.json


[{'query': 'recreation maps town',
  'relevant_datasets': ['nyc_open_data/NYCDEP Recreation Area Maps.csv'],
  'source_dataset': 'nyc_open_data/NYCDEP Recreation Area Maps.csv',
  'generation_method': 'schema+filename_tfidf'},
 {'query': 'site site type sample site',
  'relevant_datasets': ['nyc_open_data/Watershed Water Quality - Sites.csv'],
  'source_dataset': 'nyc_open_data/Watershed Water Quality - Sites.csv',
  'generation_method': 'schema+filename_tfidf'},
 {'query': 'domestic violence partners violence',
  'relevant_datasets': ['nyc_open_data/HRA Domestic Violence Partners.csv'],
  'source_dataset': 'nyc_open_data/HRA Domestic Violence Partners.csv',
  'generation_method': 'schema+filename_tfidf'},
 {'query': 'fees expenses administrative',
  'relevant_datasets': ['nyc_open_data/New York City Deferred Compensation Plan - Summary of Administrative Revenues and Expenses.csv'],
  'source_dataset': 'nyc_open_data/New York City Deferred Compensation Plan - Summary of Administrative 

In [20]:
# --- Robust corpus builder from dataset_descriptions/ JSONs (schema-agnostic) ---

import os
import json
from typing import Dict, Any, List

BUCKET_NAME = "nyu-tandon-cs-gy-6513-project"
OUTPUT_PREFIX = "dataset_descriptions/"

AUCTUS_NO_DESC_PREFIX = "auctus/dataset_no_description/"
AUCTUS_HAS_DESC_PREFIX = "auctus/dataset_has_description/"
NYC_PREFIX = "nyc_open_data/"


def list_json_keys(bucket, prefix):
    keys = []
    token = None
    while True:
        kwargs = {"Bucket": bucket, "Prefix": prefix}
        if token:
            kwargs["ContinuationToken"] = token
        resp = s3.list_objects_v2(**kwargs)
        for obj in resp.get("Contents", []):
            k = obj["Key"]
            if k.lower().endswith(".json"):
                keys.append(k)
        if resp.get("IsTruncated"):
            token = resp.get("NextContinuationToken")
        else:
            break
    return keys


def read_json(bucket, key) -> Dict[str, Any]:
    raw = s3.get_object(Bucket=bucket, Key=key)["Body"].read().decode("utf-8", errors="ignore")
    return json.loads(raw)


def extract_any_overview(obj: Dict[str, Any]) -> str:
    """
    Try many possible shapes:
    - top-level Overview/summary/description
    - nested generated
    - if obj itself is already the parsed LLM summary dict
    """
    if not isinstance(obj, dict):
        return ""

    # 1) Direct keys
    for k in ["Overview", "overview", "Summary", "summary", "Description", "description",
              "llm_overview", "generated_overview"]:
        v = obj.get(k)
        if isinstance(v, str) and v.strip():
            return v.strip()

    # 2) Nested containers
    for container_key in ["generated", "output", "result", "data"]:
        c = obj.get(container_key)
        if isinstance(c, dict):
            for k in ["Overview", "overview", "Summary", "summary", "Description", "description"]:
                v = c.get(k)
                if isinstance(v, str) and v.strip():
                    return v.strip()

    # 3) If there is a "summary" field that looks like a JSON string
    s = obj.get("summary")
    if isinstance(s, str) and s.strip():
        # Try your existing parser if available
        try:
            j = parse_llm_summary(s)
            ov = extract_overview_from_summary(j)
            if isinstance(ov, str) and ov.strip():
                return ov.strip()
        except Exception:
            # If summary is plain text, keep it as fallback
            if len(s.strip()) > 20:
                return s.strip()

    return ""


def derive_dataset_key_from_result_key(result_key: str) -> List[str]:
    """
    Given:
      dataset_descriptions/Foo_description.json
    derive likely dataset CSV keys:
      nyc_open_data/Foo.csv
      auctus/dataset_has_description/Foo.csv
      auctus/dataset_no_description/Foo.csv
    """
    base = os.path.basename(result_key)

    # Strip suffix patterns
    name = base
    if name.lower().endswith("_description.json"):
        name = name[:-len("_description.json")]
    elif name.lower().endswith(".json"):
        name = name[:-len(".json")]

    # Re-add csv extension
    fname = name + ".csv"

    return [
        NYC_PREFIX + fname,
        AUCTUS_HAS_DESC_PREFIX + fname,
        AUCTUS_NO_DESC_PREFIX + fname,
    ]


def build_augmented_corpus_from_result_jsons(
    bucket=BUCKET_NAME,
    prefix=OUTPUT_PREFIX
) -> Dict[str, str]:
    keys = list_json_keys(bucket, prefix)
    print("Result JSON count:", len(keys))

    corpus = {}

    for rk in keys:
        try:
            obj = read_json(bucket, rk)

            # Prefer explicit dataset_key if present
            dkey = obj.get("dataset_key")
            overview = extract_any_overview(obj)

            if not overview:
                continue

            if isinstance(dkey, str) and dkey.strip():
                corpus[dkey.strip()] = overview
                continue

            # Otherwise attempt to derive from filename
            candidates = derive_dataset_key_from_result_key(rk)
            # Store all candidates; downstream you can align by intersection with query_set
            for c in candidates:
                corpus.setdefault(c, overview)

        except Exception:
            continue

    return corpus


augmented_corpus = build_augmented_corpus_from_result_jsons()
print("Augmented corpus size (robust):", len(augmented_corpus))

# show a few
list(augmented_corpus.items())[:5]


Result JSON count: 1302
Augmented corpus size (robust): 1292


[('auctus/dataset_has_description/2007_-_2008_School_Progress_Reports_-_All_Schools.csv',
  '{\n  "Columns": {\n    "DBN": "text",\n    "DISTRICT": "float",\n    "SCHOOL": "text",\n    "PRINCIPAL": "text",\n    "2007-08 SCHOOL SUPPORT ORGANIZATION": "categorical",\n    "PROGRESS REPORT TYPE": "categorical",\n    "SCHOOL LEVEL*": "categorical",\n    "PEER INDEX*": "float",\n    "OVERALL GRADE": "categorical",\n    "OVERALL SCORE": "float",\n    "ENVIRONMENT CATEGORY SCORE": "float",\n    "ENVIRONMENT GRADE": "categorical",\n    "PERFORMANCE CATEGORY SCORE": "float",\n    "PERFORMANCE GRADE": "categorical",\n    "PROGRESS CATEGORY SCORE": "float",\n    "PROGRESS GRADE": "categorical",\n    "ADDITIONAL CREDIT": "float",\n    "2006-07 PROGRESS REPORT GRADE": "categorical",\n    "2007-08 QUALITY REVIEW SCORE": "categorical",\n    "2006-07 FEDERAL ACCOUNTABILITY STATUS": "categorical"\n  },\n  "Key facts": [\n    "Dataset contains many records, each representing a distinct school.",\n    "Ma

In [ ]:
# Avoids calling any previously-defined _list_s3_keys with mismatched signature.

import json
import os
from io import StringIO
import pandas as pd

BUCKET_NAME = "nyu-tandon-cs-gy-6513-project"
AUCTUS_HAS_DESC_PREFIX = "auctus/dataset_has_description/"
NYC_PREFIX = "nyc_open_data/"
NYC_CATALOG_KEY = "nyc_open_data/dataset_catalog.csv"


def list_s3_keys(bucket: str, prefix: str):
    keys = []
    token = None
    while True:
        kwargs = {"Bucket": bucket, "Prefix": prefix}
        if token:
            kwargs["ContinuationToken"] = token
        resp = s3.list_objects_v2(**kwargs)
        for obj in resp.get("Contents", []):
            keys.append(obj["Key"])
        if resp.get("IsTruncated"):
            token = resp.get("NextContinuationToken")
        else:
            break
    return keys


def read_s3_text(bucket: str, key: str) -> str:
    obj = s3.get_object(Bucket=bucket, Key=key)
    return obj["Body"].read().decode("utf-8", errors="ignore")


def read_s3_json(bucket: str, key: str):
    return json.loads(read_s3_text(bucket, key))


def extract_overview_from_any_json(d: dict) -> str:
    # handle Auctus paired JSON possibilities
    for k in ["Overview", "overview", "summary", "Summary", "description", "Description"]:
        v = d.get(k)
        if isinstance(v, str) and v.strip():
            return v.strip()

    if isinstance(d.get("generated"), dict):
        for k in ["Overview", "overview", "summary", "description"]:
            v = d["generated"].get(k)
            if isinstance(v, str) and v.strip():
                return v.strip()

    return ""


def build_auctus_baseline_map():
    keys = list_s3_keys(BUCKET_NAME, AUCTUS_HAS_DESC_PREFIX)
    key_set = set(keys)
    baseline = {}

    for csv_key in keys:
        if not csv_key.lower().endswith(".csv"):
            continue

        json_key = csv_key.rsplit(".", 1)[0] + ".json"
        if json_key not in key_set:
            continue

        try:
            meta = read_s3_json(BUCKET_NAME, json_key)
            ov = extract_overview_from_any_json(meta)
            if ov:
                baseline[csv_key] = ov
        except Exception:
            continue

    return baseline


def build_nyc_baseline_map():
    text = read_s3_text(BUCKET_NAME, NYC_CATALOG_KEY)
    df = pd.read_csv(StringIO(text), header=None)

    if df.shape[1] < 3:
        raise ValueError("dataset_catalog.csv must have >= 3 columns: id, name, summary")

    baseline = {}
    for _, row in df.iterrows():
        name = str(row[1]).strip()
        summary = str(row[2]).strip()

        if not name or not summary:
            continue

        if not name.lower().endswith(".csv"):
            name += ".csv"

        baseline[f"{NYC_PREFIX}{name}"] = summary

    return baseline


baseline_corpus = {}
baseline_corpus.update(build_auctus_baseline_map())
baseline_corpus.update(build_nyc_baseline_map())

print("Baseline corpus size (sources):", len(baseline_corpus))
list(baseline_corpus.items())[:5]


Baseline corpus size (sources): 1297


[('auctus/dataset_has_description/2007_-_2008_School_Progress_Reports_-_All_Schools.csv',
  'Description: 2007/08 Progress Report results for all schools (data as of 1/13/09) Peer indices are calculated differently depending on School Level. Schools are only compared to other schools in the same School Level (e.g., Elementary, K-8, Middle, High) 1) Elementary & K-8 - peer index is a value from 0-100. We use a composite demographic statistic based on % ELL, % SpEd, % Title I free lunch, and % Black/Hispanic. Higher values indicate student populations with higher need. 2) Middle & High - peer index is a value from 1.00-4.50. For middle schools, we use the average 4th grade proficiency ratings in ELA and Math for all their students that have 4th grade test scores. For high schools, we use the average 8th grade proficiency ratings in ELA and Math for all their students that have 8th grade test scores, % SpEd, and % Overage. Lower values indicate student populations with higher need. 3) Sch

In [22]:
# === Search impact: Offline retrieval test (robust, aligned baseline vs augmented) ===
# Uses FAISS if available; falls back to sklearn if not.
# Baseline descriptions are loaded from S3 sources:
#   - Auctus: auctus/dataset_has_description/*.json paired with CSV
#   - NYC: nyc_open_data/dataset_catalog.csv (col0 id, col1 filename, col2 summary)
# Augmented descriptions are generated Overviews from your pipeline outputs.
#
# This version aligns corpora to the set of datasets you actually generated
# so the comparison is fair and baseline isn't accidentally 0.

from typing import List, Dict, Any, Tuple, Optional
from io import StringIO
import json
import re
import numpy as np

import boto3
import pandas as pd

# -----------------------
# Safe defaults (if not defined earlier)
# -----------------------
try:
    BUCKET_NAME
except NameError:
    BUCKET_NAME = "nyu-tandon-cs-gy-6513-project"

try:
    OUTPUT_PREFIX
except NameError:
    OUTPUT_PREFIX = "dataset_descriptions/"

# If you already created s3 earlier, this won't overwrite your client variable usage
try:
    s3
except NameError:
    s3 = boto3.client("s3")


# -----------------------
# Config
# -----------------------
QUERY_SET_S3_KEY = "evaluation/query_set.json"

AUCTUS_HAS_DESC_PREFIX = "auctus/dataset_has_description/"
NYC_PREFIX = "nyc_open_data/"
NYC_CATALOG_KEY = "nyc_open_data/dataset_catalog.csv"


# -----------------------
# Optional retrieval backends
# -----------------------
USE_FAISS = False
try:
    import faiss
    USE_FAISS = True
except Exception:
    USE_FAISS = False

from sentence_transformers import SentenceTransformer


def build_index_and_search_fn(embeddings: np.ndarray):
    """
    Returns an index object and a search_fn(query_emb, k) -> (idx, scores).
    Uses cosine similarity.
    """
    embeddings = embeddings.astype("float32")

    if USE_FAISS:
        faiss.normalize_L2(embeddings)
        index = faiss.IndexFlatIP(embeddings.shape[1])
        index.add(embeddings)

        def search_fn(query_emb: np.ndarray, k: int):
            query_emb = query_emb.astype("float32")
            faiss.normalize_L2(query_emb)
            scores, idx = index.search(query_emb, k)
            return idx, scores

        return index, search_fn

    # sklearn fallback
    from sklearn.neighbors import NearestNeighbors

    norms = np.linalg.norm(embeddings, axis=1, keepdims=True) + 1e-12
    emb_norm = embeddings / norms

    nn = NearestNeighbors(metric="cosine")
    nn.fit(emb_norm)

    def search_fn(query_emb: np.ndarray, k: int):
        q = query_emb.astype("float32")
        q_norm = q / (np.linalg.norm(q, axis=1, keepdims=True) + 1e-12)
        distances, idx = nn.kneighbors(q_norm, n_neighbors=min(k, len(emb_norm)))
        scores = 1.0 - distances
        return idx, scores

    return nn, search_fn


# -----------------------
# S3 helpers
# -----------------------
def _list_s3_keys(bucket: str, prefix: str) -> List[str]:
    keys = []
    token = None
    while True:
        kwargs = {"Bucket": bucket, "Prefix": prefix}
        if token:
            kwargs["ContinuationToken"] = token
        resp = s3.list_objects_v2(**kwargs)
        for obj in resp.get("Contents", []):
            keys.append(obj["Key"])
        if resp.get("IsTruncated"):
            token = resp.get("NextContinuationToken")
        else:
            break
    return keys


def _read_s3_text(bucket: str, key: str) -> str:
    obj = s3.get_object(Bucket=bucket, Key=key)
    return obj["Body"].read().decode("utf-8", errors="ignore")


def _read_s3_json(bucket: str, key: str) -> Dict[str, Any]:
    return json.loads(_read_s3_text(bucket, key))


def _clean_text(t: str) -> str:
    t = re.sub(r"\s+", " ", str(t))
    return t.strip()


def _extract_overview_from_any_json(d: Dict[str, Any]) -> str:
    # Handles Auctus paired JSON and possible pipeline variants
    for k in ["Overview", "overview", "summary", "Summary", "description", "Description"]:
        v = d.get(k)
        if isinstance(v, str) and v.strip():
            return v.strip()

    if isinstance(d.get("generated"), dict):
        for k in ["Overview", "overview", "summary", "description"]:
            v = d["generated"].get(k)
            if isinstance(v, str) and v.strip():
                return v.strip()

    return ""


# -----------------------
# Load pipeline results
# -----------------------
def _iter_json_keys(bucket: str, prefix: str) -> List[str]:
    keys = _list_s3_keys(bucket, prefix)
    return [k for k in keys if k.lower().endswith(".json")]


def load_pipeline_results(
    bucket: str = BUCKET_NAME,
    output_prefix: str = OUTPUT_PREFIX
) -> List[Dict[str, Any]]:
    results = []
    json_keys = _iter_json_keys(bucket, output_prefix)

    for key in json_keys:
        try:
            raw = _read_s3_text(bucket, key)
            results.append(json.loads(raw))
        except Exception as e:
            print(f"Warning: could not read {key}: {e}")

    print(f"Loaded {len(results)} pipeline result JSON files from {output_prefix}")
    return results


# -----------------------
# Build baseline map from S3 sources
# -----------------------
def build_auctus_baseline_map(
    bucket: str = BUCKET_NAME,
    prefix: str = AUCTUS_HAS_DESC_PREFIX
) -> Dict[str, str]:
    """
    Map: dataset_key (CSV key) -> existing description text
    Uses paired JSON with same basename.
    """
    keys = _list_s3_keys(bucket, prefix)
    key_set = set(keys)

    csv_keys = [k for k in keys if k.lower().endswith(".csv")]
    baseline = {}

    for csv_key in csv_keys:
        json_key = csv_key.rsplit(".", 1)[0] + ".json"
        if json_key not in key_set:
            continue
        try:
            meta = _read_s3_json(bucket, json_key)
            overview = _extract_overview_from_any_json(meta)
            if overview:
                baseline[csv_key] = _clean_text(overview)
        except Exception:
            continue

    return baseline


def build_nyc_baseline_map(
    bucket: str = BUCKET_NAME,
    catalog_key: str = NYC_CATALOG_KEY,
    prefix: str = NYC_PREFIX
) -> Dict[str, str]:
    """
    NYC catalog schema:
      col 0 = id
      col 1 = name (same as filename)
      col 2 = summary
    Map: dataset_key -> summary
    """
    text = _read_s3_text(bucket, catalog_key)
    df = pd.read_csv(StringIO(text), header=None)

    if df.shape[1] < 3:
        raise ValueError("dataset_catalog.csv must have at least 3 columns: id, name, summary")

    baseline = {}
    for _, row in df.iterrows():
        name = _clean_text(row[1])
        summary = _clean_text(row[2])
        if not name or not summary:
            continue

        if not name.lower().endswith(".csv"):
            name = name + ".csv"

        dataset_key = f"{prefix}{name}"
        baseline[dataset_key] = summary

    return baseline


def build_baseline_source_map() -> Dict[str, str]:
    baseline = {}
    baseline.update(build_auctus_baseline_map())
    baseline.update(build_nyc_baseline_map())
    return baseline


# -----------------------
# Build aligned corpora
# -----------------------
def build_aligned_corpora_from_results(
    results: List[Dict[str, Any]]
) -> Tuple[Dict[str, str], Dict[str, str]]:
    """
    Returns (baseline_corpus, augmented_corpus) aligned to datasets with generated overviews.
    Baseline text pulled from S3 sources.
    """
    baseline_sources = build_baseline_source_map()

    augmented = {}
    for r in results:
        dkey = r.get("dataset_key")
        if not dkey:
            continue

        gen_sum = r.get("summary")
        try:
            # These should exist from your earlier notebook cells:
            gen_json = parse_llm_summary(gen_sum) if not isinstance(gen_sum, dict) else gen_sum
            gen_overview = extract_overview_from_summary(gen_json)
        except Exception:
            gen_overview = ""

        if gen_overview:
            augmented[dkey] = _clean_text(gen_overview)

    # Align baseline to augmented keys only
    baseline = {}
    for dkey in augmented.keys():
        txt = baseline_sources.get(dkey, "")
        if txt:
            baseline[dkey] = _clean_text(txt)

    return baseline, augmented


# -----------------------
# Load query set
# -----------------------
def load_query_set_from_s3(
    bucket: str = BUCKET_NAME,
    key: str = QUERY_SET_S3_KEY
) -> List[Dict[str, Any]]:
    try:
        raw = _read_s3_text(bucket, key)
        qs = json.loads(raw)
        print(f"Loaded query set from s3://{bucket}/{key} with {len(qs)} queries.")
        return qs
    except Exception as e:
        print(f"Could not load query set from s3://{bucket}/{key}: {e}")
        print("Falling back to a small placeholder template.")
        return [
            {"query": "public safety incidents", "relevant_datasets": []},
            {"query": "housing and rent", "relevant_datasets": []},
        ]


# -----------------------
# Evaluation
# -----------------------
def evaluate_query_set(
    corpus: Dict[str, str],
    query_set: List[Dict[str, Any]],
    model_name: str = "all-MiniLM-L6-v2",
    k: int = 5
) -> Dict[str, Any]:
    """
    Macro-averaged precision@k and recall@k across labeled queries.
    Always returns num_labeled_queries.
    """
    labeled_queries = [
        q for q in query_set
        if str(q.get("query", "")).strip() and (q.get("relevant_datasets") or [])
    ]

    if not corpus:
        return {
            "model_name": model_name,
            "k": k,
            "precision_at_k": 0.0,
            "recall_at_k": 0.0,
            "per_query": [],
            "num_labeled_queries": 0,
            "num_queries_total": len(query_set),
            "num_queries_with_labels": len(labeled_queries),
        }

    model = SentenceTransformer(model_name)

    dataset_keys = list(corpus.keys())
    texts = [corpus[k_] for k_ in dataset_keys]

    emb = model.encode(texts, convert_to_numpy=True, show_progress_bar=True)
    _, search_fn = build_index_and_search_fn(emb)

    per_query = []
    precisions = []
    recalls = []

    for q in query_set:
        query = str(q.get("query", "")).strip()
        relevant = set(q.get("relevant_datasets", []) or [])

        if not query:
            continue

        q_emb = model.encode([query], convert_to_numpy=True)
        idx, scores = search_fn(q_emb, k)

        retrieved_keys = [dataset_keys[i] for i in idx[0] if i < len(dataset_keys)]
        retrieved_set = set(retrieved_keys)

        hit_count = len(retrieved_set & relevant)

        precision = hit_count / float(k) if k else 0.0
        recall = hit_count / float(len(relevant)) if relevant else 0.0

        per_query.append({
            "query": query,
            "k": k,
            "retrieved": retrieved_keys,
            "relevant": list(relevant),
            "precision_at_k": precision,
            "recall_at_k": recall,
        })

        if relevant:
            precisions.append(precision)
            recalls.append(recall)

    precision_macro = float(np.mean(precisions)) if precisions else 0.0
    recall_macro = float(np.mean(recalls)) if recalls else 0.0

    return {
        "model_name": model_name,
        "k": k,
        "precision_at_k": precision_macro,
        "recall_at_k": recall_macro,
        "per_query": per_query,
        "num_labeled_queries": len(precisions),
        "num_queries_total": len(query_set),
        "num_queries_with_labels": len(labeled_queries),
    }

In [23]:
def filter_query_set_to_corpus(query_set, corpus):
    corpus_keys = set(corpus.keys())
    filtered = []
    for q in query_set:
        rel = set(q.get("relevant_datasets", []) or [])
        if not rel:
            continue
        # keep only queries whose relevant datasets exist in this corpus
        if rel & corpus_keys:
            filtered.append(q)
    return filtered

baseline_qs  = filter_query_set_to_corpus(query_set, baseline_corpus)
augmented_qs = filter_query_set_to_corpus(query_set, augmented_corpus)

print("Evaluable labeled queries:")
print("  Baseline :", len(baseline_qs))
print("  Augmented:", len(augmented_qs))

baseline_metrics  = evaluate_query_set(baseline_corpus,  baseline_qs,  k=5)
augmented_metrics = evaluate_query_set(augmented_corpus, augmented_qs, k=5)

print("\n=== Retrieval Impact Summary ===")
print("Baseline:")
print(json.dumps({
    "k": baseline_metrics.get("k"),
    "precision_at_k": baseline_metrics.get("precision_at_k"),
    "recall_at_k": baseline_metrics.get("recall_at_k"),
    "num_labeled_queries": baseline_metrics.get("num_labeled_queries", 0),
}, indent=2))

print("Augmented (Generated only):")
print(json.dumps({
    "k": augmented_metrics.get("k"),
    "precision_at_k": augmented_metrics.get("precision_at_k"),
    "recall_at_k": augmented_metrics.get("recall_at_k"),
    "num_labeled_queries": augmented_metrics.get("num_labeled_queries", 0),
}, indent=2))

print("\nΔ vs Baseline:")
print("  Augmented Δ Precision@k:", round(
    augmented_metrics["precision_at_k"] - baseline_metrics["precision_at_k"], 4
))
print("  Augmented Δ Recall@k:   ", round(
    augmented_metrics["recall_at_k"] - baseline_metrics["recall_at_k"], 4
))

Evaluable labeled queries:
  Baseline : 925
  Augmented: 1292


Batches:   0%|          | 0/41 [00:00<?, ?it/s]

Batches:   0%|          | 0/41 [00:00<?, ?it/s]


=== Retrieval Impact Summary ===
Baseline:
{
  "k": 5,
  "precision_at_k": 0.06616216216216217,
  "recall_at_k": 0.3308108108108108,
  "num_labeled_queries": 925
}
Augmented (Generated only):
{
  "k": 5,
  "precision_at_k": 0.09659442724458205,
  "recall_at_k": 0.48297213622291024,
  "num_labeled_queries": 1292
}

Δ vs Baseline:
  Augmented Δ Precision@k: 0.0304
  Augmented Δ Recall@k:    0.1522


Automatic Metrics

In [ ]:
from rouge_score import rouge_scorer
from evaluate import load
import json
import statistics
from tqdm.auto import tqdm 

def _read_json_from_s3(bucket: str, key: str):
    obj = s3.get_object(Bucket=bucket, Key=key)
    raw = obj["Body"].read().decode("utf-8")
    return json.loads(raw)

def compute_postrun_avg_rouge_bertscore(
    bucket_name: str = BUCKET_NAME,
    output_prefix: str = OUTPUT_PREFIX,
):
    # Collect output json keys
    keys = _list_keys(bucket_name, output_prefix)
    keys = [k for k in keys if k.lower().endswith(".json")]

    if not keys:
        print(f"No JSON outputs found under s3://{bucket_name}/{output_prefix}")
        return None

    scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)
    bertscore = load("bertscore")

    rouge_f1s = []
    bert_f1s = []
    evaluated = 0
    skipped = 0

    candidate_keys = []
    for key in keys:
        candidate_keys.append(key)

    for i, key in enumerate(tqdm(candidate_keys, desc="Post-run ROUGE/BERTScore")):
        try:
            data = _read_json_from_s3(bucket_name, key)

            # Only evaluate where we actually have an existing reference
            if data.get("run_type") != "has_existing_description":
                continue

            reference = (data.get("existing_overview") or "").strip()
            generated = extract_overview_from_summary(data.get("summary"))

            if not reference or not generated:
                skipped += 1
                continue

            # ROUGE-L F1
            r = scorer.score(reference, generated)["rougeL"].fmeasure

            # BERTScore F1
            b = bertscore.compute(
                predictions=[generated],
                references=[reference],
                lang="en",
            )
            b_f1 = float(b["f1"][0])

            rouge_f1s.append(float(r))
            bert_f1s.append(b_f1)
            evaluated += 1

            # Optional: occasional textual progress checkpoint
            if evaluated % 25 == 0:
                print(f"✓ evaluated {evaluated} so far...")

        except Exception as e:
            skipped += 1
            print(f"Skip {key} due to error: {e}")

    if evaluated == 0:
        print("No valid HAS-DESC outputs with both generated + existing Overview found.")
        return None

    avg_rouge = statistics.mean(rouge_f1s)
    avg_bert = statistics.mean(bert_f1s)

    print(f"Evaluated: {evaluated} | Skipped (missing/errored): {skipped}")
    print(f"Average ROUGE-L F1:   {avg_rouge:.4f}")
    print(f"Average BERTScore F1: {avg_bert:.4f}")

    return {
        "evaluated": evaluated,
        "skipped": skipped,
        "avg_rougeL_f1": avg_rouge,
        "avg_bertscore_f1": avg_bert,
    }

# Run it
compute_postrun_avg_rouge_bertscore()


Post-run ROUGE/BERTScore:   0%|          | 0/1302 [00:00<?, ?it/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✓ evaluated 25 so far...
✓ evaluated 50 so far...
✓ evaluated 75 so far...
✓ evaluated 100 so far...
✓ evaluated 125 so far...
✓ evaluated 150 so far...
✓ evaluated 175 so far...
✓ evaluated 200 so far...
✓ evaluated 225 so far...
✓ evaluated 250 so far...
✓ evaluated 275 so far...
✓ evaluated 300 so far...
✓ evaluated 325 so far...
✓ evaluated 350 so far...
✓ evaluated 375 so far...
✓ evaluated 400 so far...
✓ evaluated 425 so far...
✓ evaluated 450 so far...
✓ evaluated 475 so far...
✓ evaluated 500 so far...
✓ evaluated 525 so far...
✓ evaluated 550 so far...
✓ evaluated 575 so far...
✓ evaluated 600 so far...
✓ evaluated 625 so far...
✓ evaluated 650 so far...
✓ evaluated 675 so far...
✓ evaluated 700 so far...
✓ evaluated 725 so far...
✓ evaluated 750 so far...
✓ evaluated 775 so far...
✓ evaluated 800 so far...
✓ evaluated 825 so far...
✓ evaluated 850 so far...
✓ evaluated 875 so far...
Evaluated: 896 | Skipped (missing/errored): 300
Average ROUGE-L F1:   0.0959
Average BERTScor

{'evaluated': 896,
 'skipped': 300,
 'avg_rougeL_f1': 0.09590355196326665,
 'avg_bertscore_f1': 0.8108155442667859}

UI

In [ ]:

import os
import json
import io
import contextlib

import gradio as gr
import pandas as pd


def _load_dataframe_from_file(file_obj) -> pd.DataFrame:
    """Helper to load a pandas DataFrame from a Gradio File input.
    Supports CSV, Parquet, and Excel; defaults to CSV."""
    if file_obj is None:
        raise ValueError("No file uploaded.")

    if isinstance(file_obj, dict):
        path = file_obj.get("name")
    else:
        path = getattr(file_obj, "name", None)

    if path is None:
        # Fall back to reading bytes as CSV
        return pd.read_csv(io.BytesIO(file_obj.read()))

    ext = os.path.splitext(path)[1].lower()

    if ext in [".csv", ".txt"]:
        return pd.read_csv(path)
    elif ext == ".parquet":
        return pd.read_parquet(path)
    elif ext in [".xlsx", ".xls"]:
        return pd.read_excel(path)
    else:
        # Default: try CSV
        return pd.read_csv(path)


def run_full_pipeline(file_obj):
    """Run the existing pipeline on an uploaded dataset and
    return profiling, signals, summary, and evaluation outputs."""
    if file_obj is None:
        return (
            {"error": "Please upload a dataset file."},
            {},
            {},
            "No evaluation (no dataset).",
        )

    # 1. Load dataset
    df = _load_dataframe_from_file(file_obj)

    # 2. Profiling
    profile = profile_dataframe(df)

    # 3. Build signals
    # Wrap the profiler output into a "dataset" + "columns" structure
    profile_for_signals = {
        "dataset": {
            "num_rows": profile.get("num_rows"),
            "num_columns": profile.get("num_columns"),
            "geospatial_hints": profile.get("geospatial_hints"),
            "primary_key_candidates": profile.get("primary_key_candidates"),
            "type_counts": profile.get("type_counts"),
            "time_coverage": profile.get("time_coverage"),
        },
        "columns": profile.get("columns", {}),
    }
    signals = build_data_signals(profile_for_signals)

    # 4. LLM summary 
    summary_str = summarize_dataset_with_llm(
        profile,
        signals,
        side_info_text=None,
    )

    try:
        summary_json = parse_llm_summary(summary_str)
    except Exception:
        summary_json = {"raw_summary": summary_str}


    # Deterministic column/type verification
    deterministic_eval = verify_column_names_and_types_deterministic(
        ground_truth_profile=profile,
        generated_summary_str=summary_str,
    )

    # ROUGE / BERTScore
    eval_text_buf = io.StringIO()
    with contextlib.redirect_stdout(eval_text_buf):
        try:
            evaluate_summary_vs_baseline(summary_json, baseline_summary)
        except Exception as e:
            print(f"Error while computing ROUGE/BERTScore: {e}")
    rouge_bert_text = eval_text_buf.getvalue()

    # 6. LLM-based verification of the summary against the profile
    try:
        llm_verification_report = verify_summary_with_llm(
            ground_truth_profile=profile,
            generated_summary_str=summary_str,
        )
    except Exception as e:
        llm_verification_report = f"Error in LLM-based verification: {e}"

    # 7. Combine all evaluation outputs into a single text blob
    evaluation_text = (
        "=== Deterministic column/type verification ===\n"
        + json.dumps(deterministic_eval, indent=2, default=str)
        + "\n\n=== ROUGE / BERTScore evaluation ===\n"
        + (rouge_bert_text or "(no metrics computed)")
        + "\n\n=== LLM-based verification report ===\n"
        + str(llm_verification_report)
    )

    # Return:
    #  - raw profiler dict
    #  - signals dict
    #  - parsed LLM summary (JSON-like)
    #  - combined evaluation text
    return profile, signals, summary_json, evaluation_text


with gr.Blocks() as dataset_description_app:
    gr.Markdown("# LLM Dataset Description Pipeline UI")
    gr.Markdown(
        "Upload a tabular dataset to run the full pipeline: "
        "profiling → signal extraction → LLM summary → evaluation."
    )

    with gr.Row():
        file_input = gr.File(
            label="Upload dataset (CSV, Parquet, or Excel)",
            file_types=[".csv", ".txt", ".parquet", ".xlsx", ".xls"],
        )

    run_button = gr.Button("Run pipeline")

    with gr.Tab("Profiling"):
        profile_output = gr.JSON(label="Profiler output")
    with gr.Tab("Signals"):
        signals_output = gr.JSON(label="Signals output")
    with gr.Tab("LLM Summary"):
        summary_output = gr.JSON(label="Generated dataset summary")
    with gr.Tab("Evaluation"):
        evaluation_output = gr.Textbox(
            label="Evaluation outputs (deterministic checks + ROUGE/BERTScore + LLM verification)",
            lines=20,
        )

    run_button.click(
        fn=run_full_pipeline,
        inputs=[file_input],
        outputs=[profile_output, signals_output, summary_output, evaluation_output],
    )

dataset_description_app.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [ ]:
# === Compare description lengths by type (existing vs no-existing) ===
# For datasets WITH existing descriptions:
#   - compare existing_overview vs generated Overview (word count)
#   - compute average % change in length
# For datasets WITHOUT existing descriptions:
#   - report average generated Overview length only

import json
from typing import List, Dict, Any, Tuple

# Assumes these already exist:
#   - s3
#   - BUCKET_NAME
#   - OUTPUT_PREFIX = "dataset_descriptions/"
#   - parse_llm_summary
#   - extract_overview_from_summary


def _iter_json_keys_for_outputs(
    bucket: str,
    prefix: str
) -> List[str]:
    """List all JSON result keys under the pipeline output prefix."""
    keys = []
    token = None
    while True:
        kwargs = {"Bucket": bucket, "Prefix": prefix}
        if token:
            kwargs["ContinuationToken"] = token
        resp = s3.list_objects_v2(**kwargs)
        for obj in resp.get("Contents", []):
            key = obj["Key"]
            if key.lower().endswith(".json"):
                keys.append(key)
        if resp.get("IsTruncated"):
            token = resp.get("NextContinuationToken")
        else:
            break
    return keys


def load_pipeline_results_for_lengths(
    bucket: str = BUCKET_NAME,
    output_prefix: str = OUTPUT_PREFIX
) -> List[Dict[str, Any]]:
    results = []
    json_keys = _iter_json_keys_for_outputs(bucket, output_prefix)
    print(f"Found {len(json_keys)} JSON files under {output_prefix}")
    for key in json_keys:
        try:
            obj = s3.get_object(Bucket=bucket, Key=key)
            raw = obj["Body"].read().decode("utf-8")
            results.append(json.loads(raw))
        except Exception as e:
            print(f"Warning: could not read {key}: {e}")
    print(f"Loaded {len(results)} pipeline result objects.")
    return results


def _extract_generated_overview_any(summary_field: Any) -> str:
    """
    Robustly get the generated Overview text from a summary field.
    Handles:
      - already-parsed dict
      - JSON string in your LLM format
      - falls back to your extract_overview_from_summary(...)
      - if parsing fails, returns raw string
    """
    if isinstance(summary_field, dict):
        try:
            ov = extract_overview_from_summary(summary_field)
            if isinstance(ov, str) and ov.strip():
                return ov.strip()
        except Exception:
            pass
        for k in ["Overview", "overview", "summary", "Summary", "description", "Description"]:
            v = summary_field.get(k)
            if isinstance(v, str) and v.strip():
                return v.strip()
        return ""

    if isinstance(summary_field, str):
        text = summary_field.strip()
        if not text:
            return ""
        try:
            parsed = parse_llm_summary(text)
            ov = extract_overview_from_summary(parsed)
            if isinstance(ov, str) and ov.strip():
                return ov.strip()
            for k in ["Overview", "overview", "summary", "Summary", "description", "Description"]:
                v = parsed.get(k)
                if isinstance(v, str) and v.strip():
                    return v.strip()
        except Exception:
            # last resort: treat whole string as overview
            return text
    return ""


def compare_description_lengths_by_type(
    bucket: str = BUCKET_NAME,
    output_prefix: str = OUTPUT_PREFIX,
    min_words: int = 3
) -> Dict[str, Dict[str, float]]:
    """
    Returns a dict with three sub-dicts:
      {
        "has_existing": { ... },
        "no_existing": { ... },
        "overall_generated": { ... }
      }
    """
    results = load_pipeline_results_for_lengths(bucket, output_prefix)

    # For has-existing group (baseline vs generated)
    has_existing_diffs = []
    has_existing_old = []
    has_existing_new = []

    # For no-existing group (generated only)
    no_existing_new = []

    # For overall generated lengths (any run_type)
    all_generated_new = []

    for r in results:
        run_type = r.get("run_type", "").lower()
        existing = r.get("existing_overview")
        summary_field = r.get("parsed_summary", None)
        if summary_field is None:
            summary_field = r.get("summary")

        # Always try to get generated overview (for length stats)
        gen_overview = _extract_generated_overview_any(summary_field) if summary_field is not None else ""
        gen_len = len(gen_overview.split()) if gen_overview else 0
        if gen_len > 0:
            all_generated_new.append(gen_len)

        # --- Group: no existing description ---
        if run_type == "no_existing_description":
            if gen_len > 0:
                no_existing_new.append(gen_len)
            continue

        # --- Group: has existing description (only if we actually have a usable baseline) ---
        if run_type == "has_existing_description":
            if not isinstance(existing, str) or not existing.strip():
                # no baseline text available, so we treat it as "generated-only" in overall stats
                continue

            old_len = len(existing.split())
            if old_len < min_words or gen_len == 0:
                continue

            pct_change = (gen_len - old_len) / float(old_len) * 100.0

            has_existing_old.append(old_len)
            has_existing_new.append(gen_len)
            has_existing_diffs.append(pct_change)

    # Aggregate stats
    def _avg(lst):
        return sum(lst) / len(lst) if lst else 0.0

    stats = {}

    # Has-existing group
    n_has = len(has_existing_diffs)
    stats["has_existing"] = {
        "n_compared": n_has,
        "avg_existing_len": _avg(has_existing_old),
        "avg_generated_len": _avg(has_existing_new),
        "avg_pct_change": _avg(has_existing_diffs),
    }

    # No-existing group: only generated length
    n_no = len(no_existing_new)
    stats["no_existing"] = {
        "n_generated": n_no,
        "avg_generated_len": _avg(no_existing_new),
    }

    # Overall generated (any run type)
    n_all = len(all_generated_new)
    stats["overall_generated"] = {
        "n_generated": n_all,
        "avg_generated_len": _avg(all_generated_new),
    }

    # Pretty-print
    print("=== Length comparison by type ===")
    print(f"[Has existing]    n={n_has}")
    print(f"  Avg existing length (words):  {_avg(has_existing_old):.2f}")
    print(f"  Avg generated length (words): {_avg(has_existing_new):.2f}")
    print(f"  Avg % change in length:       {_avg(has_existing_diffs):+.2f}%")
    print()
    print(f"[No existing]     n_generated={n_no}")
    print(f"  Avg generated length (words): {_avg(no_existing_new):.2f}")
    print()
    print(f"[Overall]         n_generated={n_all}")
    print(f"  Avg generated length (words): {_avg(all_generated_new):.2f}")

    return stats


# ---- Run grouped comparison ----
length_stats_by_type = compare_description_lengths_by_type()
length_stats_by_type
